In [ ]:
import os
import sys
import time
import random
import shutil
import json
import csv
import warnings
from tabulate import tabulate

warnings.filterwarnings('ignore')

import h5py
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
from skimage.io import imread
from skimage.transform import resize

from sklearn.model_selection import train_test_split
from sklearn import metrics as skm
from sklearn import preprocessing as skp
from sklearn import utils as sku
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)
from sklearn.utils.class_weight import compute_class_weight

import tensorflow as tf
import tensorflow.keras.applications as applications
from tensorflow.keras import models, layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import (
    Input,
    GlobalAveragePooling2D,
    Concatenate,
    Dense,
    Dropout,
)
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, CSVLogger
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.mixed_precision import set_global_policy

from tensorflow.keras.applications import MobileNet, DenseNet121
from tensorflow.keras.applications.mobilenet import preprocess_input as mobilenet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_v2_preprocess
from tensorflow.keras.applications.densenet import preprocess_input as dense_preprocess


In [3]:
import tensorflow as tf

physical_devices = tf.config.list_physical_devices('GPU')

if physical_devices:
    print("Num GPUs Available: ", len(physical_devices))
    for gpu in physical_devices:
        print(f"  GPU Device: {gpu}")
else:
    print("No GPU devices found. TensorFlow will run on CPU.")
    print("TensorFlow version:", tf.__version__)

Num GPUs Available:  1
  GPU Device: PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')


Convert Data Mat To JPG

In [4]:
folder_pairs = [
    {
        'input': r'E:\dataset001\data',
        'output': r'E:\dataset001\datajpg'
    },
    {
        'input': r'E:\dataset001\cvind.mat',
        'output': r'E:\dataset001',
        'is_file': True  
    }
]

label_to_class = {
    1: 'meningioma',
    2: 'glioma',
    3: 'pituitary tumor'
}

def process_mat_file(file_path, output_folder):
    filename = os.path.basename(file_path)
    print(f"Processing {filename}...")

    try:
        with h5py.File(file_path, 'r') as f:
            if 'cjdata' not in f:
                print(f"Skipping {filename}: No 'cjdata' group found")
                return

            cjdata = f['cjdata']

            if 'image' not in cjdata or 'label' not in cjdata:
                print(f"Skipping {filename}: Missing 'image' or 'label' dataset")
                return

            image_array = cjdata['image'][:]

            label = cjdata['label'][()]  
            if isinstance(label, np.ndarray):
                label = label.item()  
            label = int(label) 
        if label not in label_to_class:
            print(f"Skipping {filename}: Invalid label {label}")
            return

        image_array = image_array.astype(np.float64) 
        min_val = np.min(image_array)
        max_val = np.max(image_array)
        if max_val != min_val:  
            image_array = (255 / (max_val - min_val) * (image_array - min_val)).astype(np.uint8)
        else:
            print(f"Skipping {filename}: Image has no contrast (max == min)")
            return

        if len(image_array.shape) == 2:
            image = Image.fromarray(image_array, mode='L') 
        else:
            print(f"Skipping {filename}: Unsupported array shape {image_array.shape}")
            return

        class_name = label_to_class[label]
        label_folder = os.path.join(output_folder, class_name)
        if not os.path.exists(label_folder):
            os.makedirs(label_folder)

        output_filename = os.path.splitext(filename)[0] + '.jpg'
        output_file_path = os.path.join(label_folder, output_filename)

        image.save(output_file_path, 'JPEG')
        print(f"Saved image as {output_file_path}")

    except Exception as e:
        print(f"Error processing {filename}: {str(e)}")

for pair in folder_pairs:
    input_path = pair['input']
    output_folder = pair['output']
    is_file = pair.get('is_file', False)

    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    if is_file:
        if os.path.exists(input_path) and input_path.endswith('.mat'):
            process_mat_file(input_path, output_folder)
        else:
            print(f"Skipping {input_path}: File not found or not a .mat file")
    else:
        print(f"\nProcessing folder: {input_path}")
        for filename in os.listdir(input_path):
            if filename.endswith('.mat'):
                file_path = os.path.join(input_path, filename)
                process_mat_file(file_path, output_folder)

print("Conversion complete for all folders and files!")


Processing folder: E:\dataset001\data


FileNotFoundError: [WinError 3] The system cannot find the path specified: 'E:\\dataset001\\data'

Spliting Data 90% & 
10%

ImBalanced Data


In [5]:
import os
import shutil
from sklearn.model_selection import train_test_split
from pathlib import Path

DATASETS = [
    {
        'name': 'Dataset 1',
        'input_dir': r"E:\Data001\Data",
        'output_dir': r"E:\Data001\SplitData",
        'class_names': ['Meningioma', 'Glioma', 'Pituitary']
    },
    {
        'name': 'Dataset 2',
        'input_dir': r"E:\Data002\Data",
        'output_dir': r"E:\Data002\SplitData",
        'class_names': ['Meningioma', 'Glioma', 'Pituitary', 'No-tumor']
    }
]

COMBINED_DATASET = {
    'name': 'Combined Dataset',
    'input_dir': r"E:\CombineData001_002\Data",
    'output_dir': r"E:\CombineData001_002\SplitData",
    'class_names': ['Meningioma', 'Glioma', 'Pituitary', 'No-tumor']  
}

def ensure_unique_filename(dest_path):
    """Append a suffix to the filename if it already exists."""
    dest_path = Path(dest_path)
    base, ext = dest_path.stem, dest_path.suffix
    counter = 1
    new_path = dest_path
    while new_path.exists():
        new_path = dest_path.with_name(f"{base}_{counter}{ext}")
        counter += 1
    return str(new_path)

def combine_datasets(datasets, combined_input_dir):
    """Combine multiple datasets into a single directory structure."""
    combined_input_dir = Path(combined_input_dir)
    combined_input_dir.mkdir(parents=True, exist_ok=True)
    
    for dataset in datasets:
        dataset_dir = Path(dataset['input_dir'])
        if not dataset_dir.exists():
            print(f"Error: Dataset directory {dataset_dir} does not exist.")
            continue
        
        for class_name in dataset['class_names']:
            class_path = dataset_dir / class_name
            if not class_path.is_dir():
                print(f"Warning: Class {class_name} not found in {dataset_dir}")
                continue
            
            combined_class_path = combined_input_dir / class_name
            combined_class_path.mkdir(parents=True, exist_ok=True)
            
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            for img in images:
                src = class_path / img
                dest = combined_class_path / img
                dest = ensure_unique_filename(dest)
                try:
                    shutil.copy(src, dest)
                except Exception as e:
                    print(f"Error copying {src} to {dest}: {e}")
            
            print(f"Copied {len(images)} images from {class_path} to {combined_class_path}")

def split_dataset(dataset_dir, output_dir, class_names, train_size=0.9, test_size=0.1):
    dataset_dir = Path(dataset_dir)
    output_dir = Path(output_dir)
    
    if not dataset_dir.exists():
        print(f"Error: Dataset directory {dataset_dir} does not exist.")
        return False
    if abs(train_size + test_size - 1.0) > 1e-6:
        print(f"Error: Split sizes must sum to 1. Got train={train_size}, test={test_size}.")
        return False
    
    try:
        for split in ['train', 'test']:
            for cls in class_names:
                (output_dir / split / cls).mkdir(parents=True, exist_ok=True)
    except PermissionError as e:
        print(f"Error: Cannot create directories in {output_dir}: {e}")
        return False
    
    available_classes = {d for d in os.listdir(dataset_dir) if dataset_dir.joinpath(d).is_dir()}
    missing_classes = set(class_names) - available_classes
    if missing_classes:
        print(f"Warning: Missing classes in {dataset_dir}: {missing_classes}")
    
    for class_name in available_classes:
        if class_name not in class_names:
            print(f"Skipping unknown class: {class_name}")
            continue
        
        class_path = dataset_dir / class_name
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not images:
            print(f"No images found in class: {class_name}")
            continue
        
        train_images, test_images = train_test_split(images, train_size=train_size, random_state=42)
        
        for img in train_images:
            src = class_path / img
            dest = output_dir / 'train' / class_name / img
            dest = ensure_unique_filename(dest)
            try:
                shutil.copy(src, dest)
            except Exception as e:
                print(f"Error copying {src} to {dest}: {e}")
        
        for img in test_images:
            src = class_path / img
            dest = output_dir / 'test' / class_name / img
            dest = ensure_unique_filename(dest)
            try:
                shutil.copy(src, dest)
            except Exception as e:
                print(f"Error copying {src} to {dest}: {e}")
        
        print(f"Class {class_name}: {len(train_images)} train, {len(test_images)} test")
    
    return True

def main():
    for dataset in DATASETS:
        print(f"\nSplitting {dataset['name']} ({dataset['input_dir']})...")
        success = split_dataset(
            dataset['input_dir'],
            dataset['output_dir'],
            dataset['class_names'],
            train_size=0.9,
            test_size=0.1
        )
        if success:
            print(f"{dataset['name']} split completed successfully.")
        else:
            print(f"{dataset['name']} split failed.")
    
    print(f"\nCombining datasets into {COMBINED_DATASET['input_dir']}...")
    combine_datasets(DATASETS, COMBINED_DATASET['input_dir'])
    
    print(f"\nSplitting {COMBINED_DATASET['name']} ({COMBINED_DATASET['input_dir']})...")
    success = split_dataset(
        COMBINED_DATASET['input_dir'],
        COMBINED_DATASET['output_dir'],
        COMBINED_DATASET['class_names'],
        train_size=0.9,
        test_size=0.1
    )
    if success:
        print(f"{COMBINED_DATASET['name']} split completed successfully.")
    else:
        print(f"{COMBINED_DATASET['name']} split failed.")

if __name__ == "__main__":
    main()


Splitting Dataset 1 (E:\Data001\Data)...
Class Glioma: 1283 train, 143 test
Class Pituitary: 837 train, 93 test
Class Meningioma: 637 train, 71 test
Dataset 1 split completed successfully.

Splitting Dataset 2 (E:\Data002\Data)...
Class Glioma: 833 train, 93 test
Class Pituitary: 810 train, 91 test
Class Meningioma: 843 train, 94 test
Class No-tumor: 450 train, 50 test
Dataset 2 split completed successfully.

Combining datasets into E:\CombineData001_002\Data...
Copied 708 images from E:\Data001\Data\Meningioma to E:\CombineData001_002\Data\Meningioma
Copied 1426 images from E:\Data001\Data\Glioma to E:\CombineData001_002\Data\Glioma
Copied 930 images from E:\Data001\Data\Pituitary to E:\CombineData001_002\Data\Pituitary
Copied 937 images from E:\Data002\Data\Meningioma to E:\CombineData001_002\Data\Meningioma
Copied 926 images from E:\Data002\Data\Glioma to E:\CombineData001_002\Data\Glioma
Copied 901 images from E:\Data002\Data\Pituitary to E:\CombineData001_002\Data\Pituitary
Copie

Balanced Data

In [6]:
import os
import shutil
from sklearn.model_selection import train_test_split

DATASETS = [
    {
        'name': 'Dataset 1',
        'input_dir': r"E:\Data001\Data1",
        'output_dir': r"E:\Data001\SplitData1",
        'class_names': ['Meningioma', 'Glioma', 'Pituitary']
    },
    {
        'name': 'Dataset 2',
        'input_dir': r"E:\Data002\Data1",
        'output_dir': r"E:\Data002\SplitData1",
        'class_names': ['Meningioma', 'Glioma', 'Pituitary', 'No-tumor']
    }
]

COMBINED_DATASET = {
    'name': 'Combined Dataset',
    'input_dir': r"E:\CombineData001_002\Data1",
    'output_dir': r"E:\CombineData001_002\SplitData1",
    'class_names': ['Meningioma', 'Glioma', 'Pituitary', 'No-tumor']  
}

def ensure_unique_filename(dest_path):
    """Append a suffix to the filename if it already exists."""
    dest_path = Path(dest_path)
    base, ext = dest_path.stem, dest_path.suffix
    counter = 1
    new_path = dest_path
    while new_path.exists():
        new_path = dest_path.with_name(f"{base}_{counter}{ext}")
        counter += 1
    return str(new_path)

def combine_datasets(datasets, combined_input_dir):
    """Combine multiple datasets into a single directory structure."""
    combined_input_dir = Path(combined_input_dir)
    combined_input_dir.mkdir(parents=True, exist_ok=True)
    
    for dataset in datasets:
        dataset_dir = Path(dataset['input_dir'])
        if not dataset_dir.exists():
            print(f"Error: Dataset directory {dataset_dir} does not exist.")
            continue
        
        for class_name in dataset['class_names']:
            class_path = dataset_dir / class_name
            if not class_path.is_dir():
                print(f"Warning: Class {class_name} not found in {dataset_dir}")
                continue
            
            combined_class_path = combined_input_dir / class_name
            combined_class_path.mkdir(parents=True, exist_ok=True)
            
            images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
            for img in images:
                src = class_path / img
                dest = combined_class_path / img
                dest = ensure_unique_filename(dest)
                try:
                    shutil.copy(src, dest)
                except Exception as e:
                    print(f"Error copying {src} to {dest}: {e}")
            
            print(f"Copied {len(images)} images from {class_path} to {combined_class_path}")

def split_dataset(dataset_dir, output_dir, class_names, train_size=0.9, test_size=0.1):
    dataset_dir = Path(dataset_dir)
    output_dir = Path(output_dir)
    
    if not dataset_dir.exists():
        print(f"Error: Dataset directory {dataset_dir} does not exist.")
        return False
    if abs(train_size + test_size - 1.0) > 1e-6:
        print(f"Error: Split sizes must sum to 1. Got train={train_size}, test={test_size}.")
        return False
    
    try:
        for split in ['train', 'test']:
            for cls in class_names:
                (output_dir / split / cls).mkdir(parents=True, exist_ok=True)
    except PermissionError as e:
        print(f"Error: Cannot create directories in {output_dir}: {e}")
        return False
    
    available_classes = {d for d in os.listdir(dataset_dir) if dataset_dir.joinpath(d).is_dir()}
    missing_classes = set(class_names) - available_classes
    if missing_classes:
        print(f"Warning: Missing classes in {dataset_dir}: {missing_classes}")
    
    for class_name in available_classes:
        if class_name not in class_names:
            print(f"Skipping unknown class: {class_name}")
            continue
        
        class_path = dataset_dir / class_name
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
        if not images:
            print(f"No images found in class: {class_name}")
            continue
        
        train_images, test_images = train_test_split(images, train_size=train_size, random_state=42)
        
        for img in train_images:
            src = class_path / img
            dest = output_dir / 'train' / class_name / img
            dest = ensure_unique_filename(dest)
            try:
                shutil.copy(src, dest)
            except Exception as e:
                print(f"Error copying {src} to {dest}: {e}")
        
        for img in test_images:
            src = class_path / img
            dest = output_dir / 'test' / class_name / img
            dest = ensure_unique_filename(dest)
            try:
                shutil.copy(src, dest)
            except Exception as e:
                print(f"Error copying {src} to {dest}: {e}")
        
        print(f"Class {class_name}: {len(train_images)} train, {len(test_images)} test")
    
    return True

def main():
    for dataset in DATASETS:
        print(f"\nSplitting {dataset['name']} ({dataset['input_dir']})...")
        success = split_dataset(
            dataset['input_dir'],
            dataset['output_dir'],
            dataset['class_names'],
            train_size=0.9,
            test_size=0.1
        )
        if success:
            print(f"{dataset['name']} split completed successfully.")
        else:
            print(f"{dataset['name']} split failed.")
    
    print(f"\nCombining datasets into {COMBINED_DATASET['input_dir']}...")
    combine_datasets(DATASETS, COMBINED_DATASET['input_dir'])
    
    print(f"\nSplitting {COMBINED_DATASET['name']} ({COMBINED_DATASET['input_dir']})...")
    success = split_dataset(
        COMBINED_DATASET['input_dir'],
        COMBINED_DATASET['output_dir'],
        COMBINED_DATASET['class_names'],
        train_size=0.9,
        test_size=0.1
    )
    if success:
        print(f"{COMBINED_DATASET['name']} split completed successfully.")
    else:
        print(f"{COMBINED_DATASET['name']} split failed.")

if __name__ == "__main__":
    main()


Splitting Dataset 1 (E:\Data001\Data1)...
Error: Dataset directory E:\Data001\Data1 does not exist.
Dataset 1 split failed.

Splitting Dataset 2 (E:\Data002\Data1)...
Error: Dataset directory E:\Data002\Data1 does not exist.
Dataset 2 split failed.

Combining datasets into E:\CombineData001_002\Data1...
Error: Dataset directory E:\Data001\Data1 does not exist.
Error: Dataset directory E:\Data002\Data1 does not exist.

Splitting Combined Dataset (E:\CombineData001_002\Data1)...


KeyboardInterrupt: 

In [2]:
class NumpyEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.floating):
            return float(obj)
        if isinstance(obj, np.integer):
            return int(obj)
        if isinstance(obj, np.ndarray):
            return obj.tolist()
        return super(NumpyEncoder, self).default(obj)

def compute_metrics(y_true, y_pred, dataset_name):
    precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    return {
        f"{dataset_name}_precision": precision,
        f"{dataset_name}_recall": recall,
        f"{dataset_name}_f1": f1
    }
def evaluate_model(hybrid_model, test_generator, class_names, counter, results_dir, model_name):
    start_time = time.time()
    test_loss, test_acc = hybrid_model.evaluate(test_generator, verbose=0)
    y_true = test_generator.classes
    y_pred = np.argmax(hybrid_model.predict(test_generator, verbose=0), axis=1)
    y_probs = hybrid_model.predict(test_generator, verbose=0)
    inference_time = (time.time() - start_time) * 1000

    y_probs = np.nan_to_num(y_probs / np.sum(y_probs, axis=1, keepdims=True), nan=0.0, posinf=1.0, neginf=0.0)
    test_precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
    test_recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
    test_f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    auc = roc_auc_score(y_true, y_probs, multi_class='ovr') if len(np.unique(y_true)) > 1 else None
    auc_display = f"{auc:.4f}" if auc is not None else 'N/A'

    print(f"Test Results for {model_name}: Loss={test_loss:.4f}, Acc={test_acc:.4f}, "
          f"Precision={test_precision:.4f}, Recall={test_recall:.4f}, F1={test_f1:.4f}, "
          f"AUC={auc_display}, Inference Time={inference_time:.2f}ms")

    return {
        'Model with input image size': model_name,
        'Accuracy': test_acc,
        'AUC': auc,
        'Loss': test_loss,
        'Precision': test_precision,
        'Recall': test_recall,
        'F1 Score': test_f1,
        'Inference time (in miliseconds)': inference_time
    }, y_true, y_pred

def plot_training_history(history, results_dir, model_name, counter, class_names, best_y_true, best_y_pred):
    plt.figure(figsize=(18, 6), dpi=300)
    
    plt.subplot(1, 3, 1)
    plt.plot(history['accuracy'], label='Train Accuracy', color='#1f77b4', linewidth=2)
    plt.plot(history['val_accuracy'], label='Test Accuracy', color='#ff7f0e', linestyle='--', linewidth=2)
    plt.title('Train vs Test Accuracy', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Accuracy', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    plt.ylim(0, 1.05)
    
    plt.subplot(1, 3, 2)
    plt.plot(history['loss'], label='Train Loss', color='#1f77b4', linewidth=2)
    plt.plot(history['val_loss'], label='Test Loss', color='#ff7f0e', linestyle='--', linewidth=2)
    plt.title('Train vs Test Loss', fontsize=14)
    plt.xlabel('Epoch', fontsize=12)
    plt.ylabel('Loss', fontsize=12)
    plt.legend(fontsize=10)
    plt.grid(True, alpha=0.3)
    
    plt.subplot(1, 3, 3)
    cm = confusion_matrix(best_y_true, best_y_pred, labels=range(len(class_names)))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names, cbar=False, square=True)
    plt.title('Test Confusion Matrix', fontsize=14)
    plt.xlabel('Predicted', fontsize=12)
    plt.ylabel('True', fontsize=12)
    plt.xticks(rotation=45, ha='right', fontsize=10)
    
    plt.tight_layout()
    output_path = os.path.join(results_dir, f'{model_name}_training_metrics_{counter}.png')
    plt.savefig(output_path, dpi=300, bbox_inches='tight')
    plt.close()

    loss_chart = {
        "type": "line",
        "data": {
            "labels": [str(i+1) for i in range(len(history['loss']))],
            "datasets": [
                {"label": "Train Loss", "data": history['loss'], "borderColor": "rgba(31, 119, 180, 1)", "fill": False},
                {"label": "Test Loss", "data": history['val_loss'], "borderColor": "rgba(255, 127, 14, 1)", "fill": False, "borderDash": [5, 5]}
            ]
        },
        "options": {
            "responsive": True,
            "plugins": {"title": {"display": True, "text": "Train vs Test Loss"}},
            "scales": {"x": {"title": {"display": True, "text": "Epoch"}}, "y": {"title": {"display": True, "text": "Loss"}}}
        }
    }
    accuracy_chart = {
        "type": "line",
        "data": {
            "labels": [str(i+1) for i in range(len(history['accuracy']))],
            "datasets": [
                {"label": "Train Accuracy", "data": history['accuracy'], "borderColor": "rgba(31, 119, 180, 1)", "fill": False},
                {"label": "Test Accuracy", "data": history['val_accuracy'], "borderColor": "rgba(255, 127, 14, 1)", "fill": False, "borderDash": [5, 5]}
            ]
        },
        "options": {
            "responsive": True,
            "plugins": {"title": {"display": True, "text": "Train vs Test Accuracy"}},
            "scales": {"x": {"title": {"display": True, "text": "Epoch"}}, "y": {"title": {"display": True, "text": "Accuracy"}, "max": 1}}
        }
    }
    with open(os.path.join(results_dir, f'{model_name}_loss_chart_{counter}.json'), 'w') as f:
        json.dump(loss_chart, f, cls=NumpyEncoder, indent=2)
    with open(os.path.join(results_dir, f'{model_name}_accuracy_chart_{counter}.json'), 'w') as f:
        json.dump(accuracy_chart, f, cls=NumpyEncoder, indent=2)

Data Generator for Imbalance Data

In [8]:
# Define parameters
target_size = (224, 224)
batch_size = 16
EPOCHS_INITIAL = 5

# Define dataset directories
imbalance_dataset1_train_dir = r'E:\Data001\SplitData\train'
imbalance_dataset1_test_dir = r'E:\Data001\SplitData\test'
imbalance_dataset2_train_dir = r'E:\Data002\SplitData\train'
imbalance_dataset2_test_dir = r'E:\Data002\SplitData\test'
imbalance_combined_train_dir = r'E:\CombineData001_002\SplitData\train'
imbalance_combined_test_dir = r'E:\CombineData001_002\SplitData\test'

# Data augmentation for training
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Dataset001 Generators
imbalance_train_generator1 = datagen.flow_from_directory(
    directory=imbalance_dataset1_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator1 = datagen.flow_from_directory(
    directory=imbalance_dataset1_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Dataset002 Generators
imbalance_train_generator2 = datagen.flow_from_directory(
    directory=imbalance_dataset2_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator2 = datagen.flow_from_directory(
    directory=imbalance_dataset2_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# CombinedDataset Generators
imbalance_train_generator_combined = datagen.flow_from_directory(
    directory=imbalance_combined_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator_combined = datagen.flow_from_directory(
    directory=imbalance_combined_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Calculate total images using generator .n attribute
dataset1 = imbalance_train_generator1.n + imbalance_test_generator1.n
dataset2 = imbalance_train_generator2.n + imbalance_test_generator2.n
combineddataset = imbalance_train_generator_combined.n + imbalance_test_generator_combined.n

print(f"Total dataset1 images: {dataset1}")
print(f"Total dataset2 images: {dataset2}")
print(f"Total dataset Combined images: {combineddataset}")

Found 5514 images belonging to 3 classes.
Found 614 images belonging to 3 classes.
Found 5872 images belonging to 4 classes.
Found 656 images belonging to 4 classes.
Found 17082 images belonging to 4 classes.
Found 1902 images belonging to 4 classes.
Total dataset1 images: 6128
Total dataset2 images: 6528
Total dataset Combined images: 18984


In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

target_size = (224, 224)
batch_size = 16
EPOCHS_INITIAL = 5

balance_dataset1_train_dir = r'E:\Data001\SplitData1\train'
balance_dataset1_test_dir = r'E:\Data001\SplitData1\test'
balance_dataset2_train_dir = r'E:\Data002\SplitData1\train'
balance_dataset2_test_dir = r'E:\Data002\SplitData1\test'
balance_combined_train_dir = r'E:\CombineData001_002\SplitData1\train'
balance_combined_test_dir = r'E:\CombineData001_002\SplitData1\test'

datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Dataset001 Generators
balance_train_generator1 = datagen.flow_from_directory(
    directory=balance_dataset1_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator1 = datagen.flow_from_directory(
    directory=balance_dataset1_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Dataset002 Generators
balance_train_generator2 = datagen.flow_from_directory(
    directory=balance_dataset2_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator2 = datagen.flow_from_directory(
    directory=balance_dataset2_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# CombinedDataset Generators
balance_train_generator_combined = datagen.flow_from_directory(
    directory=balance_combined_train_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator_combined = datagen.flow_from_directory(
    directory=balance_combined_test_dir,
    target_size=target_size,
    batch_size=batch_size,
    class_mode='categorical',
    shuffle=False
)

# Calculate total images using generator .n attribute
dataset1 = balance_train_generator1.n + balance_test_generator1.n
dataset2 = balance_train_generator2.n + balance_test_generator2.n
combineddataset = balance_train_generator_combined.n + balance_test_generator_combined.n

print(f"Total dataset1 images: {dataset1}")
print(f"Total dataset2 images: {dataset2}")
print(f"Total dataset Combined images: {combineddataset}")

Found 1911 images belonging to 3 classes.
Found 213 images belonging to 3 classes.
Found 1800 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Found 4446 images belonging to 4 classes.
Found 413 images belonging to 4 classes.
Total dataset1 images: 2124
Total dataset2 images: 2000
Total dataset Combined images: 4859


Build Model

In [6]:
def build_mobdensenet(
    num_classes,
    input_shape=(224, 224, 3),
    model_name="MobDenseNet",
    activation_function="relu",
    kernel_initializer="glorot_uniform",
    output_activation="softmax",
    output_kernel_initializer="random_uniform",
    dropout_rate=0.2,
    learning_rate=0.002,
    optimizer="nadam",
):

    raw_input = Input(shape=input_shape)

    mob_model = applications.MobileNet(
        input_shape=input_shape,
        alpha=1.0,
        include_top=False,
        weights="imagenet",
        pooling=None
    )
    mob_out = mob_model(raw_input)  
    mob_gap = GlobalAveragePooling2D()(mob_out)

    dense_model = applications.DenseNet121(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling=None
    )
    dense_out = dense_model(raw_input)
    dense_gap = GlobalAveragePooling2D()(dense_out)


    combined = Concatenate()([mob_gap, dense_gap])
    dropout = Dropout(dropout_rate)(combined)
    output = Dense(
        num_classes,
        activation=output_activation,
        kernel_initializer=output_kernel_initializer
    )(dropout)

    model = Model(inputs=raw_input, outputs=output, name=f"{model_name}_{num_classes}class")

    print(f"\nModel Summary for {num_classes}-Class Dataset:")
    model.summary()

    print(f"\nHyper-Parameters ({num_classes}-Class Model):")
    print("| Hyper-Parameter Name        | Value            |")
    print("|----------------------------|------------------|")
    print(f"| Activation Function         | {activation_function}              |")
    print(f"| Kernel Initializer          | {kernel_initializer}   |")
    print(f"| Initial Learning Rate       | {learning_rate}            |")
    print(f"| Optimizer                   | {optimizer}            |")
    print(f"| Batch Size                  | {batch_size}               |")
    print(f"| Epochs                      | {EPOCHS_INITIAL}               |")
    print(f"| Train-test Split            | {train_test_split}           |")
    print(f"| Output Activation Function  | {output_activation}            |")
    print(f"| Output Kernel Initializer   | {output_kernel_initializer} |")
    print(f"| Number of Output Classes    | {num_classes}                |")
    print(f"| Dropout Rate                | {dropout_rate}               |")

    return model

Verify Dataset

In [12]:
# Calculate steps per epoch and validation steps for each dataset
steps_per_epoch1 = max(1, imbalance_train_generator1.samples // batch_size)
validation_steps1 = max(1, imbalance_test_generator1.samples // batch_size)

steps_per_epoch2 = max(1, imbalance_train_generator2.samples // batch_size)
validation_steps2 = max(1, imbalance_test_generator2.samples // batch_size)

steps_per_epoch3 = max(1, imbalance_train_generator_combined.samples // batch_size)
validation_steps3 = max(1, imbalance_test_generator_combined.samples // batch_size)

steps_per_epoch4 = max(1, balance_train_generator1.samples // batch_size)
validation_steps4 = max(1, balance_test_generator1.samples // batch_size)

steps_per_epoch5 = max(1, balance_train_generator2.samples // batch_size)
validation_steps5 = max(1, balance_test_generator2.samples // batch_size)

steps_per_epoch6 = max(1, balance_train_generator_combined.samples // batch_size)
validation_steps6 = max(1, balance_test_generator_combined.samples // batch_size)

table_data = [
    ["Imbalance Dataset 1", imbalance_train_generator1.samples, imbalance_test_generator1.samples,
     steps_per_epoch1, validation_steps1],
    ["Imbalance Dataset 2", imbalance_train_generator2.samples, imbalance_test_generator2.samples,
     steps_per_epoch2, validation_steps2],
    ["Imbalance Combined Dataset", imbalance_train_generator_combined.samples, imbalance_test_generator_combined.samples,
     steps_per_epoch3, validation_steps3],
    ["Balanced Dataset 1", balance_train_generator1.samples, balance_test_generator1.samples,
     steps_per_epoch4, validation_steps4],
    ["Balanced Dataset 2", balance_train_generator2.samples, balance_test_generator2.samples,
     steps_per_epoch5, validation_steps5],
    ["Balanced Combined Dataset", balance_train_generator_combined.samples, balance_test_generator_combined.samples,
     steps_per_epoch6, validation_steps6]
]

headers = ["Dataset", "Training Samples", "Validation Samples", "Steps per Epoch", "Validation Steps"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Dataset                    |   Training Samples |   Validation Samples |   Steps per Epoch |   Validation Steps |
+============================+====================+======================+===================+====================+
| Imbalance Dataset 1        |               5514 |                  614 |               344 |                 38 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Imbalance Dataset 2        |               5872 |                  656 |               367 |                 41 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Imbalance Combined Dataset |              17082 |                 1902 |              1067 |                118 |
+----------------------------+--------------------+---------------------

Train Dataset 1 with Plotting

In [ ]:
import os
import time
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from tabulate import tabulate
import tensorflow as tf

# Define dataset configurations
datasets = [
    {
        "name": "Imbalance Dataset 1",
        "train_generator": imbalance_train_generator1,
        "test_generator": imbalance_test_generator1,
        "steps_per_epoch": max(1, imbalance_train_generator1.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator1.samples // batch_size),
        "output_dir": r"E:\Data001\SplitData\hybrid_MobDenseNet_Output_using_tensorflow\imbalanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary'],
        "num_classes": 3
    },
    {
        "name": "Imbalance Dataset 2",
        "train_generator": imbalance_train_generator2,
        "test_generator": imbalance_test_generator2,
        "steps_per_epoch": max(1, imbalance_train_generator2.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator2.samples // batch_size),
        "output_dir": r"E:\Data002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\imbalanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary','No-tumor'],
        "num_classes": 4
    },
    {
        "name": "Imbalance Combined Dataset",
        "train_generator": imbalance_train_generator_combined,
        "test_generator": imbalance_test_generator_combined,
        "steps_per_epoch": max(1, imbalance_train_generator_combined.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator_combined.samples // batch_size),
        "output_dir": r"E:\CombineData001_002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\imbalanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary','No-tumor'],
        "num_classes": 4
    },
    {
        "name": "Balanced Dataset 1",
        "train_generator": balance_train_generator1,
        "test_generator": balance_test_generator1,
        "steps_per_epoch": max(1, balance_train_generator1.samples // batch_size),
        "validation_steps": max(1, balance_test_generator1.samples // batch_size),
        "output_dir": r"E:\Data001\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary'],
        "num_classes": 3
    },
    {
        "name": "Balanced Dataset 2",
        "train_generator": balance_train_generator2,
        "test_generator": balance_test_generator2,
        "steps_per_epoch": max(1, balance_train_generator2.samples // batch_size),
        "validation_steps": max(1, balance_test_generator2.samples // batch_size),
        "output_dir": r"E:\Data002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary','No-tumor'],
        "num_classes": 4
    },
    {
        "name": "Balanced Combined Dataset",
        "train_generator": balance_train_generator_combined,
        "test_generator": balance_test_generator_combined,
        "steps_per_epoch": max(1, balance_train_generator_combined.samples // batch_size),
        "validation_steps": max(1, balance_test_generator_combined.samples // batch_size),
        "output_dir": r"E:\CombineData001_002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": ['Glioma', 'Meningioma', 'Pituitary','No-tumor'],
        "num_classes": 4
    }
]

for dataset in datasets:
    if not hasattr(dataset["train_generator"], "samples") or not hasattr(dataset["test_generator"], "samples"):
        raise ValueError(f"Generator for {dataset['name']} is missing 'samples' attribute.")
    if dataset["train_generator"].samples == 0 or dataset["test_generator"].samples == 0:
        raise ValueError(f"{dataset['name']} has zero samples in train or test generator.")

# Display dataset sizes and steps
table_data = [
    [d["name"], d["train_generator"].samples, d["test_generator"].samples, 
     d["steps_per_epoch"], d["validation_steps"], d["num_classes"]]
    for d in datasets
]
headers = ["Dataset", "Training Samples", "Validation Samples", "Steps per Epoch", "Validation Steps", "Num Classes"]
print("\nDataset Sizes and Steps:")
print(tabulate(table_data, headers=headers, tablefmt="grid"))

# Model parameters
MODEL_NAME = "mobdensenet"
input_shape = (224, 224, 3)
input_image_size = f"{input_shape[0]}x{input_shape[1]}" 
EPOCHS_INITIAL = 100  # Defined as 100, but can use 5 if preferred

# Callbacks
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('hybrid_model_best.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, min_lr=0.002)
]

# Function to plot training history and confusion matrix
def plot_training_history(history, output_dir, model_name, counter, class_names, y_true, y_pred):
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy', color='#1f77b4')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy', color='#ff7f0e')
    plt.title('Training and Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss', color='#2ca02c')
    plt.plot(history.history['val_loss'], label='Validation Loss', color='#d62728')
    plt.title('Training and Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{model_name}_training_validation_{counter}.png'))
    plt.close()

    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.savefig(os.path.join(output_dir, f'{model_name}_confusion_matrix_{counter}.png'))
    plt.close()

# Function to save Chart.js config
def save_chartjs_configs(history, output_dir, model_name, counter):
    chart_config = {
        "type": "line",
        "data": {
            "labels": list(range(1, len(history.history['accuracy']) + 1)),
            "datasets": [
                {"label": "Training Accuracy", "data": history.history['accuracy'], 
                 "borderColor": "rgba(31, 119, 180, 1)", "fill": False},
                {"label": "Validation Accuracy", "data": history.history['val_accuracy'], 
                 "borderColor": "rgba(255, 127, 14, 1)", "fill": False},
                {"label": "Training Loss", "data": history.history['loss'], 
                 "borderColor": "rgba(44, 160, 44, 1)", "fill": False},
                {"label": "Validation Loss", "data": history.history['val_loss'], 
                 "borderColor": "rgba(214, 39, 40, 1)", "fill": False}
            ]
        },
        "options": {
            "scales": {
                "x": {"title": {"display": True, "text": "Epoch"}},
                "y": {"title": {"display": True, "text": "Value"}}
            },
            "plugins": {"title": {"display": True, "text": f"{model_name} Training History"}}
        }
    }
    with open(os.path.join(output_dir, f'{model_name}_chartjs_{counter}.json'), 'w') as f:
        json.dump(chart_config, f, indent=4)

# Train and evaluate for each dataset
results_summary = []
for counter, dataset in enumerate(datasets, 1):
    dataset_name = dataset["name"]
    train_generator = dataset["train_generator"]
    test_generator = dataset["test_generator"]
    steps_per_epoch = dataset["steps_per_epoch"]
    validation_steps = dataset["validation_steps"]
    output_dir = dataset["output_dir"]
    class_names = dataset["class_names"]
    num_classes = dataset["num_classes"]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    print(f"\nTraining on {dataset_name}...")
    # Build and compile a fresh model for each dataset
    model = build_mobdensenet(num_classes=num_classes, input_shape=input_shape)
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model
    history = model.fit(
        train_generator,
        steps_per_epoch=steps_per_epoch,
        validation_data=test_generator,
        validation_steps=validation_steps,
        epochs=EPOCHS_INITIAL,
        callbacks=callbacks
    )

    # Save training history
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(output_dir, f'{MODEL_NAME}_history_{counter}.csv'), index=False)

    # Evaluate on test set
    test_generator.reset()
    start_time = time.time()
    predictions = model.predict(test_generator, steps=test_generator.samples // batch_size + 1)
    inference_time = (time.time() - start_time) * 1000
    y_pred = np.argmax(predictions, axis=1)
    y_true = test_generator.classes

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)
    try:
        auc = roc_auc_score(test_generator.classes, predictions, multi_class='ovr')
    except ValueError as e:
        auc = "N/A"
        print(f"Warning: AUC calculation failed for {dataset_name}: {str(e)}")

    test_loss, test_accuracy = model.evaluate(test_generator)

    # Print classification report
    print(f"\nClassification Report for {dataset_name}:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Save metrics
    metrics = {
        'precision_per_class': precision.tolist(),
        'recall_per_class': recall.tolist(),
        'f1_per_class': f1.tolist(),
        'auc': auc,
        'accuracy': test_accuracy
    }
    with open(os.path.join(output_dir, f'{MODEL_NAME}_metrics_{counter}.json'), 'w') as f:
        json.dump(metrics, f, indent=4)

    # Calculate average metrics
    precision_avg = np.mean(metrics['precision_per_class'])
    recall_avg = np.mean(metrics['recall_per_class'])
    f1_avg = np.mean(metrics['f1_per_class'])

    # Save results to CSV
    results = [{
        'Model with input image size': f"{MODEL_NAME} {input_image_size}",
        'Dataset': dataset_name,
        'Accuracy': test_accuracy,
        'AUC': auc if auc != "N/A" else 0.0,
        'Loss': test_loss,
        'Precision': precision_avg,
        'Recall': recall_avg,
        'F1 Score': f1_avg,
        'Inference time (in miliseconds)': inference_time
    }]
    results_summary.append(results[0])
    with open(os.path.join(output_dir, f'{MODEL_NAME}_results_{counter}.csv'), 'w', newline='') as csvfile:
        fieldnames = ['Model with input image size', 'Dataset', 'Accuracy', 'AUC', 'Loss', 'Precision', 'Recall', 'F1 Score', 'Inference time (in miliseconds)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(results[0])

    # Plot and save training history and confusion matrix
    plot_training_history(history, output_dir, MODEL_NAME, counter, class_names, y_true, y_pred)
    save_chartjs_configs(history, output_dir, MODEL_NAME, counter)

    # Save the model
    model.save(os.path.join(output_dir, f'{MODEL_NAME}_{counter}.h5'))
    print(f"Model saved successfully to {os.path.join(output_dir, f'{MODEL_NAME}_{counter}.h5')}")

    # Clear session to free memory
    tf.keras.backend.clear_session()

    print(f"[{dataset_name}] Processing completed.")

# Display summary table of results
print("\nSummary of Results Across Datasets:")
summary_table = [
    [r['Dataset'], f"{r['Accuracy']:.4f}", r['AUC'] if r['AUC'] != "N/A" else "N/A", 
     f"{r['Loss']:.4f}", f"{r['Precision']:.4f}", f"{r['Recall']:.4f}", 
     f"{r['F1 Score']:.4f}", f"{r['Inference time (in miliseconds)']:.2f}"]
    for r in results_summary
]
summary_headers = ["Dataset", "Accuracy", "AUC", "Loss", "Precision", "Recall", "F1 Score", "Inference Time (ms)"]
print(tabulate(summary_table, headers=summary_headers, tablefmt="grid"))

# Save summary table to CSV
summary_output_dir = r"E:\CombineData001_002\SplitData\hybrid_MobDenseNet_Output_using_tensorflow"
os.makedirs(summary_output_dir, exist_ok=True)
with open(os.path.join(summary_output_dir, 'all_datasets_results_summary.csv'), 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=summary_headers)
    writer.writeheader()
    for row in results_summary:
        writer.writerow({
            'Dataset': row['Dataset'],
            'Accuracy': f"{row['Accuracy']:.4f}",
            'AUC': row['AUC'] if row['AUC'] != "N/A" else "N/A",
            'Loss': f"{row['Loss']:.4f}",
            'Precision': f"{row['Precision']:.4f}",
            'Recall': f"{row['Recall']:.4f}",
            'F1 Score': f"{row['F1 Score']:.4f}",
            'Inference Time (ms)': f"{row['Inference time (in miliseconds)']:.2f}"
        })

# Optional: Bar chart comparing accuracy across datasets (uncomment to enable)
"""
accuracy_data = [r['Accuracy'] for r in results_summary]
dataset_names = [r['Dataset'] for r in results_summary]
chart_config = {
    "type": "bar",
    "data": {
        "labels": dataset_names,
        "datasets": [{
            "label": "Test Accuracy",
            "data": accuracy_data,
            "backgroundColor": "rgba(31, 119, 180, 0.7)",
            "borderColor": "rgba(31, 119, 180, 1)",
            "borderWidth": 1
        }]
    },
    "options": {
        "scales": {
            "x": {"title": {"display": True, "text": "Dataset"}},
            "y": {"title": {"display": True, "text": "Accuracy"}, "beginAtZero": True, "max": 1}
        },
        "plugins": {"title": {"display": True, "text": "Test Accuracy Across Datasets"}}
    }
}
with open(os.path.join(summary_output_dir, 'accuracy_comparison_chart.json'), 'w') as f:
    json.dump(chart_config, f, indent=4)
"""

print("\nTraining and evaluation completed for all datasets.")


Dataset Sizes and Steps:
+----------------------------+--------------------+----------------------+-------------------+--------------------+---------------+
| Dataset                    |   Training Samples |   Validation Samples |   Steps per Epoch |   Validation Steps |   Num Classes |
+============================+====================+======================+===================+====================+===============+
| Imbalance Dataset 1        |               2757 |                  307 |               172 |                 19 |             3 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+---------------+
| Imbalance Dataset 2        |               2936 |                  328 |               183 |                 20 |             4 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+---------------+
| Imbalance Combined Dataset |               5693 

CBAM + MOBDENSENET

In [15]:
# Define parameters
target_size_cbam = (128, 128)
batch_size_cbam = 16
EPOCHS_INITIAL = 5

# Define dataset directories
imbalance_dataset1_train_dir = r'E:\Data001\SplitData\train'
imbalance_dataset1_test_dir = r'E:\Data001\SplitData\test'
imbalance_dataset2_train_dir = r'E:\Data002\SplitData\train'
imbalance_dataset2_test_dir = r'E:\Data002\SplitData\test'
imbalance_combined_train_dir = r'E:\CombineData001_002\SplitData\train'
imbalance_combined_test_dir = r'E:\CombineData001_002\SplitData\test'

# Data augmentation for training
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Dataset001 Generators
imbalance_train_generator1_cbam = datagen.flow_from_directory(
    directory=imbalance_dataset1_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator1_cbam = datagen.flow_from_directory(
    directory=imbalance_dataset1_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# Dataset002 Generators
imbalance_train_generator2_cbam = datagen.flow_from_directory(
    directory=imbalance_dataset2_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator2_cbam = datagen.flow_from_directory(
    directory=imbalance_dataset2_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# CombinedDataset Generators
imbalance_train_generator_combined_cbam = datagen.flow_from_directory(
    directory=imbalance_combined_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

imbalance_test_generator_combined_cbam = datagen.flow_from_directory(
    directory=imbalance_combined_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# Calculate total images using generator .n attribute
dataset1 = imbalance_train_generator1_cbam.n + imbalance_test_generator1_cbam.n
dataset2 = imbalance_train_generator2_cbam.n + imbalance_test_generator2_cbam.n
combineddataset = imbalance_train_generator_combined_cbam.n + imbalance_test_generator_combined_cbam.n

print(f"Total dataset1 images: {dataset1}")
print(f"Total dataset2 images: {dataset2}")
print(f"Total dataset Combined images: {combineddataset}")

Found 5514 images belonging to 3 classes.
Found 614 images belonging to 3 classes.
Found 5872 images belonging to 4 classes.
Found 656 images belonging to 4 classes.
Found 17082 images belonging to 4 classes.
Found 1902 images belonging to 4 classes.
Total dataset1 images: 6128
Total dataset2 images: 6528
Total dataset Combined images: 18984


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define parameters
target_size_cbam = (128, 128)
batch_size_cbam = 16
EPOCHS_INITIAL = 5

# Define dataset directories
balance_dataset1_train_dir = r'E:\Data001\SplitData1\train'
balance_dataset1_test_dir = r'E:\Data001\SplitData1\test'
balance_dataset2_train_dir = r'E:\Data002\SplitData1\train'
balance_dataset2_test_dir = r'E:\Data002\SplitData1\test'
balance_combined_train_dir = r'E:\CombineData001_002\SplitData1\train'
balance_combined_test_dir = r'E:\CombineData001_002\SplitData1\test'

# Data augmentation for training
datagen = ImageDataGenerator(
    rescale=1/255,
    rotation_range=15,
    width_shift_range=0.2,
    height_shift_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode='nearest'
)

# Dataset001 Generators
balance_train_generator1_cbam = datagen.flow_from_directory(
    directory=balance_dataset1_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator1_cbam = datagen.flow_from_directory(
    directory=balance_dataset1_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# Dataset002 Generators
balance_train_generator2_cbam = datagen.flow_from_directory(
    directory=balance_dataset2_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator2_cbam = datagen.flow_from_directory(
    directory=balance_dataset2_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# CombinedDataset Generators
balance_train_generator_combined_cbam = datagen.flow_from_directory(
    directory=balance_combined_train_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=True
)

balance_test_generator_combined_cbam = datagen.flow_from_directory(
    directory=balance_combined_test_dir,
    target_size=target_size_cbam,
    batch_size=batch_size_cbam,
    class_mode='categorical',
    shuffle=False
)

# Calculate total images using generator .n attribute
dataset1 = balance_train_generator1_cbam.n + balance_test_generator1_cbam.n
dataset2 = balance_test_generator1_cbam.n + balance_test_generator2_cbam.n
combineddataset = balance_train_generator_combined_cbam.n + balance_test_generator_combined_cbam.n

print(f"Total dataset1 images: {dataset1}")
print(f"Total dataset2 images: {dataset2}")
print(f"Total dataset Combined images: {combineddataset}")

Found 1911 images belonging to 3 classes.
Found 213 images belonging to 3 classes.
Found 1800 images belonging to 4 classes.
Found 200 images belonging to 4 classes.
Found 4446 images belonging to 4 classes.
Found 413 images belonging to 4 classes.
Total dataset1 images: 2124
Total dataset2 images: 413
Total dataset Combined images: 4859


In [17]:
# Calculate steps per epoch and validation steps for each dataset
steps_per_epoch1 = max(1, imbalance_train_generator1_cbam.samples // batch_size)
validation_steps1 = max(1, imbalance_test_generator1_cbam.samples // batch_size)

steps_per_epoch2 = max(1, imbalance_train_generator2_cbam.samples // batch_size)
validation_steps2 = max(1, imbalance_test_generator2_cbam.samples // batch_size)

steps_per_epoch3 = max(1, imbalance_train_generator_combined_cbam.samples // batch_size)
validation_steps3 = max(1, imbalance_test_generator_combined_cbam.samples // batch_size)

steps_per_epoch4 = max(1, balance_train_generator1_cbam.samples // batch_size)
validation_steps4 = max(1, balance_test_generator1_cbam.samples // batch_size)

steps_per_epoch5 = max(1, balance_train_generator2_cbam.samples // batch_size)
validation_steps5 = max(1, balance_test_generator2_cbam.samples // batch_size)

steps_per_epoch6 = max(1, balance_train_generator_combined_cbam.samples // batch_size)
validation_steps6 = max(1, balance_test_generator_combined_cbam.samples // batch_size)

table_data = [
    ["Imbalance Dataset 1", imbalance_train_generator1_cbam.samples, imbalance_test_generator1_cbam.samples,
     steps_per_epoch1, validation_steps1],
    ["Imbalance Dataset 2", imbalance_train_generator2_cbam.samples, imbalance_test_generator2_cbam.samples,
     steps_per_epoch2, validation_steps2],
    ["Imbalance Combined Dataset", imbalance_train_generator_combined_cbam.samples, imbalance_test_generator_combined_cbam.samples,
     steps_per_epoch3, validation_steps3],
    ["Balanced Dataset 1", balance_train_generator1_cbam.samples, balance_test_generator1_cbam.samples,
     steps_per_epoch4, validation_steps4],
    ["Balanced Dataset 2", balance_train_generator2_cbam.samples, balance_test_generator2_cbam.samples,
     steps_per_epoch5, validation_steps5],
    ["Balanced Combined Dataset", balance_train_generator_combined_cbam.samples, balance_test_generator_combined_cbam.samples,
     steps_per_epoch6, validation_steps6]
]

headers = ["Dataset", "Training Samples", "Validation Samples", "Steps per Epoch", "Validation Steps"]
print(tabulate(table_data, headers=headers, tablefmt="grid"))

+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Dataset                    |   Training Samples |   Validation Samples |   Steps per Epoch |   Validation Steps |
+============================+====================+======================+===================+====================+
| Imbalance Dataset 1        |               5514 |                  614 |               344 |                 38 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Imbalance Dataset 2        |               5872 |                  656 |               367 |                 41 |
+----------------------------+--------------------+----------------------+-------------------+--------------------+
| Imbalance Combined Dataset |              17082 |                 1902 |              1067 |                118 |
+----------------------------+--------------------+---------------------

In [ ]:
import os
import time
import json
import csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, classification_report, confusion_matrix
from tabulate import tabulate
import tensorflow as tf
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, GlobalMaxPooling2D, Concatenate, Multiply, Reshape, Conv2D, Input, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras import applications
import cv2
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array
import tensorflow as tf
from tensorflow.keras import applications, layers, models, mixed_precision
from tensorflow.keras.layers import (
    Input, Dense, Dropout, Conv2D, BatchNormalization, GlobalAveragePooling2D,
    GlobalMaxPooling2D, Multiply, Reshape, Concatenate
)
from tensorflow.keras.models import Model

# ==================== GPU MEMORY OPTIMIZATION ====================
# Configure GPU memory growth to prevent OOM errors
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Enable memory growth
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        print(f"✓ GPU memory growth enabled for {len(gpus)} GPU(s)")
    except RuntimeError as e:
        print(f"✗ GPU configuration error: {e}")

# Enable mixed precision training (reduces memory usage by ~50%)
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)
print(" Mixed precision training enabled")

# ==================== USE YOUR EXISTING PARAMETERS ====================
# Use the parameters you defined earlier
batch_size = 16
input_shape = (128, 128, 3)
EPOCHS_INITIAL = 5
print(f" Using your parameters: batch_size={batch_size}, input_shape={input_shape}, EPOCHS={EPOCHS_INITIAL}")

# ==================== CBAM BLOCK ====================
def cbam_block(input_feature, ratio=8, kernel_size=7, name=None):
    avg_pool = GlobalAveragePooling2D()(input_feature)
    max_pool = GlobalMaxPooling2D()(input_feature)
    
    channel = input_feature.shape[-1]
    shared_layer_one = Dense(channel // ratio, activation='relu', kernel_initializer='he_normal')
    shared_layer_two = Dense(channel, kernel_initializer='he_normal')
    
    avg_out = shared_layer_two(shared_layer_one(avg_pool))
    max_out = shared_layer_two(shared_layer_one(max_pool))
    
    channel_attention = tf.keras.layers.Add()([avg_out, max_out])
    channel_attention = tf.keras.layers.Activation('sigmoid')(channel_attention)
    channel_attention = Reshape((1, 1, channel))(channel_attention)
    channel_refined = Multiply()([input_feature, channel_attention])
    
    avg_pool_spatial = tf.reduce_mean(channel_refined, axis=-1, keepdims=True)
    max_pool_spatial = tf.reduce_max(channel_refined, axis=-1, keepdims=True)
    spatial_attention = Concatenate(axis=-1)([avg_pool_spatial, max_pool_spatial])
    spatial_attention = Conv2D(1, kernel_size, padding='same', activation='sigmoid', kernel_initializer='he_normal')(spatial_attention)
    spatial_refined = Multiply()([channel_refined, spatial_attention])
    
    if name:
        spatial_refined = tf.keras.layers.Lambda(lambda x: x, name=name)(spatial_refined)
    return spatial_refined

# ==================== LIGHTWEIGHT MODEL BUILDING ====================
def build_mobdensenet(
    num_classes,
    input_shape=input_shape,
    model_name="MobDenseNet_CBAM",
    activation_function="relu",
    kernel_initializer="glorot_uniform",
    output_activation="softmax",
    output_kernel_initializer="random_uniform",
    dropout_rate=0.2,
    learning_rate=0.002,
    optimizer="nadam",
):
    raw_input = Input(shape=input_shape)

    # Use smaller MobileNet version
    mob_model = applications.MobileNet(
        input_shape=input_shape,
        alpha=0.75,  # Reduced from 1.0 to save memory
        include_top=False,
        weights="imagenet",
        pooling=None
    )
    mob_out = mob_model(raw_input)
    mob_cbam = cbam_block(mob_out, name="cbam_mobilenet")

    # Use smaller DenseNet version
    dense_model = applications.DenseNet121(
        include_top=False,
        weights="imagenet",
        input_shape=input_shape,
        pooling=None
    )
    dense_out = dense_model(raw_input)
    dense_cbam = cbam_block(dense_out, name="cbam_densenet")


    combined_features = Concatenate(axis=-1)([mob_cbam, dense_cbam])
    
    conv_block = Conv2D(256, (3, 3), padding="same", activation="relu", name="gradcam_conv2d")(combined_features)
    conv_block = BatchNormalization()(conv_block)
    gap = GlobalAveragePooling2D()(conv_block)
    dropout = Dropout(dropout_rate)(gap)
    
    # Use mixed precision compatible output layer
    output = Dense(
        num_classes,
        activation=output_activation,
        kernel_initializer=output_kernel_initializer,
        dtype='float32'  # Ensure output is float32 for stability
    )(dropout)

    model = Model(inputs=raw_input, outputs=output, name=f"{model_name}_{num_classes}class")

    print(f"\nModel Summary for {num_classes}-Class Dataset:")
    model.summary()

    print(f"\nHyper-Parameters ({num_classes})-Class Model):")
    print("| Hyper-Parameter Name        | Value            |")
    print("|----------------------------|------------------|")
    print(f"| Activation Function         | {activation_function}              |")
    print(f"| Kernel Initializer          | {kernel_initializer}   |")
    print(f"| Initial Learning Rate       | {learning_rate}            |")
    print(f"| Optimizer                   | {optimizer}            |")
    print(f"| Output Activation Function  | {output_activation}            |")
    print(f"| Output Kernel Initializer   | {output_kernel_initializer} |")
    print(f"| Number of Output Classes    | {num_classes}                |")
    print(f"| Dropout Rate                | {dropout_rate}               |")

    return model

# ==================== DATASET CONFIGURATION ====================
# Use your existing generators (remove the _cbam suffix since they don't exist)
datasets = [
    {
        "name": "Imbalanced Dataset 1",
        "train_generator": imbalance_train_generator1_cbam,
        "test_generator": imbalance_test_generator1_cbam,
        "steps_per_epoch": max(1, imbalance_train_generator1_cbam.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator1_cbam.samples // batch_size),
        "output_dir": r"E:\Data001\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow",
        "class_names": list(imbalance_train_generator1_cbam.class_indices.keys()),
        "num_classes": len(imbalance_train_generator1_cbam.class_indices)
    },
    {
        "name": "Imbalanced Dataset 2",
        "train_generator": imbalance_train_generator2_cbam,
        "test_generator": imbalance_test_generator2_cbam,
        "steps_per_epoch": max(1, imbalance_train_generator2_cbam.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator2_cbam.samples // batch_size),
        "output_dir": r"E:\Data002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow",
        "class_names": list(imbalance_train_generator2_cbam.class_indices.keys()),
        "num_classes": len(imbalance_train_generator2_cbam.class_indices)
    },
    {
        "name": "Imbalanced Combined Dataset",
        "train_generator": imbalance_train_generator_combined_cbam,
        "test_generator": imbalance_test_generator_combined_cbam,
        "steps_per_epoch": max(1, imbalance_train_generator_combined_cbam.samples // batch_size),
        "validation_steps": max(1, imbalance_test_generator_combined_cbam.samples // batch_size),
        "output_dir": r"E:\CombineData001_002\SplitData\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow",
        "class_names": list(imbalance_train_generator_combined_cbam.class_indices.keys()),
        "num_classes": len(imbalance_train_generator_combined_cbam.class_indices)
    },
    {
        "name": "Balanced Dataset 1",
        "train_generator": balance_train_generator1_cbam,
        "test_generator": balance_test_generator1_cbam,
        "steps_per_epoch": max(1, balance_train_generator1_cbam.samples // batch_size),
        "validation_steps": max(1, balance_test_generator1_cbam.samples // batch_size),
        "output_dir": r"E:\Data001\SplitData1\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": list(balance_train_generator1_cbam.class_indices.keys()),
        "num_classes": len(balance_train_generator1_cbam.class_indices)
    },
    {
        "name": "Balanced Dataset 2",
        "train_generator": balance_train_generator2_cbam,
        "test_generator": balance_test_generator2_cbam,
        "steps_per_epoch": max(1, balance_train_generator2_cbam.samples // batch_size),
        "validation_steps": max(1, balance_test_generator2_cbam.samples // batch_size),
        "output_dir": r"E:\Data002\SplitData1\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": list(balance_train_generator2_cbam.class_indices.keys()),
        "num_classes": len(balance_train_generator2_cbam.class_indices)
    },
    {
        "name": "Balanced Combined Dataset",
        "train_generator": balance_train_generator_combined_cbam,
        "test_generator": balance_test_generator_combined_cbam,
        "steps_per_epoch": max(1, balance_train_generator_combined_cbam.samples // batch_size),
        "validation_steps": max(1, balance_test_generator_combined_cbam.samples // batch_size),
        "output_dir": r"E:\CombineData001_002\SplitData1\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput",
        "class_names": list(balance_train_generator_combined_cbam.class_indices.keys()),
        "num_classes": len(balance_train_generator_combined_cbam.class_indices)
    }
]

# Verify dataset generators
for dataset in datasets:
    if not hasattr(dataset["train_generator"], "samples") or not hasattr(dataset["test_generator"], "samples"):
        raise ValueError(f"Generator for {dataset['name']} is missing 'samples' attribute.")
    if dataset["train_generator"].samples == 0 or dataset["test_generator"].samples == 0:
        raise ValueError(f"{dataset['name']} has zero samples in train or test generator.")

# Display dataset sizes and steps
table_data = [
    [d["name"], d["train_generator"].samples, d["test_generator"].samples, 
     d["steps_per_epoch"], d["validation_steps"], d["num_classes"]]
    for d in datasets
]
headers = ["Dataset", "Training Samples", "Validation Samples", "Steps per Epoch", "Validation Steps", "Num Classes"]
print("\nDataset Sizes and Steps:")
print(tabulate(table_data, headers=headers, tablefmt="grid"))

MODEL_NAME = "mobdensenet_cbam"
input_image_size = f"{input_shape[0]}x{input_shape[1]}"

# Callbacks  memory optimization
callbacks = [
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True),
    tf.keras.callbacks.ModelCheckpoint('hybrid_model_best.h5', monitor='val_loss', save_best_only=True),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=15, min_lr=0.00002)
]

# ==================== TRAINING FUNCTIONS ====================
def plot_training_history(history, output_dir, model_name, counter, class_names, y_true, y_pred):
    # Create subplots
    fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 5))
    
    # Plot accuracy
    ax1.plot(history.history['accuracy'], label='Training Accuracy', color='#1f77b4', linewidth=2)
    ax1.plot(history.history['val_accuracy'], label='Validation Accuracy', color='#ff7f0e', linewidth=2)
    ax1.set_title('Training and Validation Accuracy', fontsize=14, fontweight='bold')
    ax1.set_xlabel('Epoch', fontsize=12)
    ax1.set_ylabel('Accuracy', fontsize=12)
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Plot loss
    ax2.plot(history.history['loss'], label='Training Loss', color='#2ca02c', linewidth=2)
    ax2.plot(history.history['val_loss'], label='Validation Loss', color='#d62728', linewidth=2)
    ax2.set_title('Training and Validation Loss', fontsize=14, fontweight='bold')
    ax2.set_xlabel('Epoch', fontsize=12)
    ax2.set_ylabel('Loss', fontsize=12)
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{model_name}_training_validation_{counter}.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                cbar_kws={'shrink': 0.8})
    plt.title('Confusion Matrix', fontsize=16, fontweight='bold')
    plt.xlabel('Predicted Labels', fontsize=12)
    plt.ylabel('True Labels', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{model_name}_confusion_matrix_{counter}.png'), dpi=300, bbox_inches='tight')
    plt.close()

    # Additional plots: Precision-Recall and F1-Score per class
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)
    
    # Precision-Recall bar plot
    fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(18, 6))
    
    x = np.arange(len(class_names))
    width = 0.25
    
    ax1.bar(x - width, precision, width, label='Precision', alpha=0.8)
    ax1.bar(x, recall, width, label='Recall', alpha=0.8)
    ax1.bar(x + width, f1, width, label='F1-Score', alpha=0.8)
    ax1.set_xlabel('Classes')
    ax1.set_title('Precision, Recall, and F1-Score by Class')
    ax1.set_xticks(x)
    ax1.set_xticklabels(class_names, rotation=45, ha='right')
    ax1.legend()
    ax1.grid(True, alpha=0.3)
    
    # Training history detailed
    ax2.plot(history.history['accuracy'], label='Train Accuracy', color='blue', linestyle='-')
    ax2.plot(history.history['val_accuracy'], label='Val Accuracy', color='blue', linestyle='--')
    ax2.plot(history.history['loss'], label='Train Loss', color='red', linestyle='-')
    ax2.plot(history.history['val_loss'], label='Val Loss', color='red', linestyle='--')
    ax2.set_title('Training History Detailed')
    ax2.set_xlabel('Epoch')
    ax2.legend()
    ax2.grid(True, alpha=0.3)
    
    # Class distribution
    class_counts = np.bincount(y_true)
    ax3.bar(range(len(class_counts)), class_counts, alpha=0.7)
    ax3.set_title('Class Distribution in Test Set')
    ax3.set_xlabel('Class Index')
    ax3.set_ylabel('Count')
    ax3.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, f'{model_name}_detailed_metrics_{counter}.png'), dpi=300, bbox_inches='tight')
    plt.close()

def save_chartjs_configs(history, output_dir, model_name, counter):
    try:
        loss_chart = {
            "type": "line",
            "data": {
                "labels": [str(i+1) for i in range(len(history.history['loss']))],
                "datasets": [
                    {
                        "label": "Train Loss",
                        "data": history.history['loss'],
                        "borderColor": "rgba(31, 119, 180, 1)",
                        "backgroundColor": "rgba(31, 119, 180, 0.2)",
                        "fill": False,
                        "tension": 0.4
                    },
                    {
                        "label": "Test Loss",
                        "data": history.history['val_loss'],
                        "borderColor": "rgba(255, 127, 14, 1)",
                        "backgroundColor": "rgba(255, 127, 14, 0.2)",
                        "fill": False,
                        "borderDash": [5, 5]
                    }
                ]
            },
            "options": {
                "responsive": True,
                "plugins": {
                    "legend": {"position": "top"},
                    "title": {"display": True, "text": f"{model_name} Train vs Test Loss"}
                },
                "scales": {
                    "x": {"title": {"display": True, "text": "Epoch"}},
                    "y": {"title": {"display": True, "text": "Loss"}, "beginAtZero": True}
                }
            }
        }
        accuracy_chart = {
            "type": "line",
            "data": {
                "labels": [str(i+1) for i in range(len(history.history['accuracy']))],
                "datasets": [
                    {
                        "label": "Train Accuracy",
                        "data": history.history['accuracy'],
                        "borderColor": "rgba(31, 119, 180, 1)",
                        "backgroundColor": "rgba(31, 119, 180, 0.2)",
                        "fill": False,
                        "tension": 0.4
                    },
                    {
                        "label": "Test Accuracy",
                        "data": history.history['val_accuracy'],
                        "borderColor": "rgba(255, 127, 14, 1)",
                        "backgroundColor": "rgba(255, 127, 14, 0.2)",
                        "fill": False,
                        "borderDash": [5, 5]
                    }
                ]
            },
            "options": {
                "responsive": True,
                "plugins": {
                    "legend": {"position": "top"},
                    "title": {"display": True, "text": f"{model_name} Train vs Test Accuracy"}
                },
                "scales": {
                    "x": {"title": {"display": True, "text": "Epoch"}},
                    "y": {"title": {"display": True, "text": "Accuracy"}, "beginAtZero": True, "max": 1}
                }
            }
        }
        with open(os.path.join(output_dir, f'{model_name}_loss_chart_{counter}.json'), 'w') as f:
            json.dump(loss_chart, f, indent=4)
        with open(os.path.join(output_dir, f'{model_name}_accuracy_chart_{counter}.json'), 'w') as f:
            json.dump(accuracy_chart, f, indent=4)
    except KeyError as e:
        print(f"Error in save_chartjs_configs: Missing key {e}. Available keys: {list(history.history.keys())}")

# ==================== MAIN TRAINING LOOP ====================
results_summary = []
for counter, dataset in enumerate(datasets, 1):
    dataset_name = dataset["name"]
    train_generator = dataset["train_generator"]
    test_generator = dataset["test_generator"]
    steps_per_epoch = dataset["steps_per_epoch"]
    validation_steps = dataset["validation_steps"]
    output_dir = dataset["output_dir"]
    class_names = dataset["class_names"]
    num_classes = dataset["num_classes"]

    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)

    print(f"\n{'='*60}")
    print(f"TRAINING ON {dataset_name}")
    print(f"{'='*60}")
    
    # Build and compile a fresh model for each dataset
    model = build_mobdensenet(num_classes=num_classes, input_shape=input_shape)
    
    # Use Adam optimizer with mixed precision compatibility
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-4)
    model.compile(
        optimizer=optimizer,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )

    # Train the model with memory monitoring
    try:
        print(f"Starting training with batch_size={batch_size}, input_size={input_shape}")
        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            validation_data=test_generator,
            validation_steps=validation_steps,
            epochs=EPOCHS_INITIAL,
            callbacks=callbacks,
            verbose=1
        )
        print("✓ Training completed successfully!")

    except tf.errors.ResourceExhaustedError as e:
        print(f"✗ OOM Error during training: {e}")
        print("Trying with even smaller batch size...")
        
        # Further reduce batch size for this dataset
        train_generator.batch_size = 8
        test_generator.batch_size = 8
        steps_per_epoch = max(1, train_generator.samples // 8)
        validation_steps = max(1, test_generator.samples // 8)
        
        history = model.fit(
            train_generator,
            steps_per_epoch=steps_per_epoch,
            validation_data=test_generator,
            validation_steps=validation_steps,
            epochs=EPOCHS_INITIAL,
            callbacks=callbacks,
            verbose=1
        )

    # Save training history
    history_df = pd.DataFrame(history.history)
    history_df.to_csv(os.path.join(output_dir, f'{MODEL_NAME}_history_{counter}.csv'), index=False)

    # Evaluate on test set
    test_generator.reset()
    start_time = time.time()
    
    # Collect all predictions
    predictions = []
    y_true = test_generator.classes
    total_samples = test_generator.samples
    steps = int(np.ceil(total_samples / batch_size))
    
    for i in range(steps):
        try:
            x_batch, _ = next(test_generator)
            batch_pred = model.predict(x_batch, verbose=0)
            predictions.append(batch_pred)
        except StopIteration:
            break
    
    predictions = np.concatenate(predictions, axis=0)
    predictions = predictions[:total_samples]
    y_pred = np.argmax(predictions, axis=1)
    
    inference_time = (time.time() - start_time) * 1000

    if len(y_pred) != len(y_true):
        print(f"Warning: Mismatch in y_pred ({len(y_pred)}) and y_true ({len(y_true)}) for {dataset_name}")
        continue

    # Calculate metrics
    precision = precision_score(y_true, y_pred, average=None)
    recall = recall_score(y_true, y_pred, average=None)
    f1 = f1_score(y_true, y_pred, average=None)
    try:
        auc = roc_auc_score(y_true, predictions, multi_class='ovr')
    except ValueError as e:
        auc = "N/A"
        print(f"Warning: AUC calculation failed for {dataset_name}: {str(e)}")

    test_loss, test_accuracy = model.evaluate(test_generator, steps=steps, verbose=0)

    # Print classification report
    print(f"\nClassification Report for {dataset_name}:")
    print(classification_report(y_true, y_pred, target_names=class_names))

    # Save metrics
    metrics = {
        'precision_per_class': precision.tolist(),
        'recall_per_class': recall.tolist(),
        'f1_per_class': f1.tolist(),
        'auc': auc,
        'accuracy': test_accuracy
    }
    with open(os.path.join(output_dir, f'{MODEL_NAME}_metrics_{counter}.json'), 'w') as f:
        json.dump(metrics, f, indent=4)

    # Calculate average metrics
    precision_avg = np.mean(metrics['precision_per_class'])
    recall_avg = np.mean(metrics['recall_per_class'])
    f1_avg = np.mean(metrics['f1_per_class'])

    # Save results to CSV
    results = [{
        'Model with input image size': f"{MODEL_NAME} {input_image_size}",
        'Dataset': dataset_name,
        'Accuracy': test_accuracy,
        'AUC': auc if auc != "N/A" else 0.0,
        'Loss': test_loss,
        'Precision': precision_avg,
        'Recall': recall_avg,
        'F1 Score': f1_avg,
        'Inference time (in miliseconds)': inference_time
    }]
    results_summary.append(results[0])
    
    with open(os.path.join(output_dir, f'{MODEL_NAME}_results_{counter}.csv'), 'w', newline='') as csvfile:
        fieldnames = ['Model with input image size', 'Dataset', 'Accuracy', 'AUC', 'Loss', 'Precision', 'Recall', 'F1 Score', 'Inference time (in miliseconds)']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        writer.writerow(results[0])

    # Plot and save training history and confusion matrix
    plot_training_history(history, output_dir, MODEL_NAME, counter, class_names, y_true, y_pred)
    save_chartjs_configs(history, output_dir, MODEL_NAME, counter)

    # Save the model
    model_path = os.path.join(output_dir, f'{MODEL_NAME}_{counter}.h5')
    model.save(model_path)
    print(f"✓ Model saved successfully to {model_path}")

    # Clear session to free memory
    tf.keras.backend.clear_session()
    print(f"✓ Memory cleared for next dataset")

    print(f"[{dataset_name}] Processing completed.")

# Display summary table
print("\nSummary of Results Across Datasets:")
summary_table = [
    [r['Dataset'], f"{r['Accuracy']:.4f}", r['AUC'] if r['AUC'] != "N/A" else "N/A", 
     f"{r['Loss']:.4f}", f"{r['Precision']:.4f}", f"{r['Recall']:.4f}", 
     f"{r['F1 Score']:.4f}", f"{r['Inference time (in miliseconds)']:.2f}"]
    for r in results_summary
]
summary_headers = ["Dataset", "Accuracy", "AUC", "Loss", "Precision", "Recall", "F1 Score", "Inference Time (ms)"]
print(tabulate(summary_table, headers=summary_headers, tablefmt="grid"))

# Save summary table to CSV
summary_output_dir = r"E:\CombineData001_002\SplitData1\hybrid_MobDenseNet_Output_using_tensorflow"
os.makedirs(summary_output_dir, exist_ok=True)
with open(os.path.join(summary_output_dir, 'all_datasets_results_summary.csv'), 'w', newline='') as csvfile:
    writer = csv.DictWriter(csvfile, fieldnames=summary_headers)
    writer.writeheader()
    for row in results_summary:
        writer.writerow({
            'Dataset': row['Dataset'],
            'Accuracy': f"{row['Accuracy']:.4f}",
            'AUC': row['AUC'] if row['AUC'] != "N/A" else "N/A",
            'Loss': f"{row['Loss']:.4f}",
            'Precision': f"{row['Precision']:.4f}",
            'Recall': f"{row['Recall']:.4f}",
            'F1 Score': f"{row['F1 Score']:.4f}",
            'Inference Time (ms)': f"{row['Inference time (in miliseconds)']:.2f}"
        })

print("✓ All datasets processed successfully!")

✓ GPU memory growth enabled for 1 GPU(s)
 Mixed precision training enabled
 Using your parameters: batch_size=16, input_shape=(128, 128, 3), EPOCHS=5

Dataset Sizes and Steps:
+-----------------------------+--------------------+----------------------+-------------------+--------------------+---------------+
| Dataset                     |   Training Samples |   Validation Samples |   Steps per Epoch |   Validation Steps |   Num Classes |
+=============================+====================+======================+===================+====================+===============+
| Imbalanced Dataset 1        |               5514 |                  614 |               344 |                 38 |             3 |
+-----------------------------+--------------------+----------------------+-------------------+--------------------+---------------+
| Imbalanced Dataset 2        |               5872 |                  656 |               367 |                 41 |             4 |
+-------------------------

UnknownError: Graph execution error:

2 root error(s) found.
  (0) UNKNOWN:  UnidentifiedImageError: cannot identify image file <_io.BytesIO object at 0x000001C9DF4C6C00>
Traceback (most recent call last):

  File "e:\.env\lib\site-packages\tensorflow\python\ops\script_ops.py", line 271, in __call__
    ret = func(*args)

  File "e:\.env\lib\site-packages\tensorflow\python\autograph\impl\api.py", line 642, in wrapper
    return func(*args, **kwargs)

  File "e:\.env\lib\site-packages\tensorflow\python\data\ops\dataset_ops.py", line 1035, in generator_py_func
    values = next(generator_state.get_iterator(iterator_id))

  File "e:\.env\lib\site-packages\keras\engine\data_adapter.py", line 903, in wrapped_generator
    for data in generator_fn():

  File "e:\.env\lib\site-packages\keras\engine\data_adapter.py", line 1050, in generator_fn
    yield x[i]

  File "e:\.env\lib\site-packages\keras\preprocessing\image.py", line 116, in __getitem__
    return self._get_batches_of_transformed_sa

In [14]:
# ==================== DETECTION BOX FUNCTIONS ====================
def create_gradcam_bounding_box(img_path, heatmap, output_path, threshold=0.5):
    """Create bounding boxes based on Grad-CAM heatmap activation"""
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, input_shape[:2])
    
    # Resize heatmap
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    
    # Threshold heatmap to create binary mask
    _, thresh = cv2.threshold(heatmap, threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    
    # Find contours
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding boxes
    for contour in contours:
        if cv2.contourArea(contour) > 10:  # Filter small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green for Grad-CAM
    
    # Save image with bounding boxes
    cv2.imwrite(output_path, img)
    return img

def create_traditional_bounding_box(img_path, output_path):
    """Create bounding boxes using traditional image processing methods"""
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, input_shape[:2])
    img_copy = img.copy()
    
    # Convert to grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # Apply multiple detection methods
    
    # Method 1: Edge detection
    edges = cv2.Canny(gray, 50, 150)
    
    # Method 2: Thresholding
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # Method 3: Morphological operations to clean up
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    
    # Combine methods
    combined = cv2.bitwise_or(edges, morph)
    
    # Find contours
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw bounding boxes
    for contour in contours:
        if cv2.contourArea(contour) > 50:  # Filter small contours
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue for traditional
    
    # Save image with bounding boxes
    cv2.imwrite(output_path, img_copy)
    return img_copy

def create_combined_bounding_boxes(img_path, heatmap, output_path, gradcam_threshold=0.5):
    """Create image with both Grad-CAM and traditional bounding boxes"""
    # Load image
    img = cv2.imread(img_path)
    img = cv2.resize(img, input_shape[:2])
    img_copy = img.copy()
    
    # Grad-CAM bounding boxes (Green)
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    _, thresh = cv2.threshold(heatmap_resized, gradcam_threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    contours_gradcam, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours_gradcam:
        if cv2.contourArea(contour) > 10:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)  # Green
    
    # Traditional bounding boxes (Blue)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    _, thresh_trad = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh_trad, cv2.MORPH_CLOSE, kernel)
    combined = cv2.bitwise_or(edges, morph)
    contours_trad, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours_trad:
        if cv2.contourArea(contour) > 50:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)  # Blue
    
    # Add legend
    cv2.putText(img_copy, "Grad-CAM Boxes (Green)", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(img_copy, "Traditional Boxes (Blue)", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    cv2.imwrite(output_path, img_copy)
    return img_copy

# ==================== MODIFIED VISUALIZATION FUNCTION ====================
def visualize_detection_boxes(model, test_generator, output_dir, dataset_name, counter, num_samples=5):
    """Visualize both Grad-CAM and traditional detection boxes"""
    detection_dir = os.path.join(output_dir, 'detection_boxes')
    os.makedirs(detection_dir, exist_ok=True)
    
    # Get sample images
    test_generator.reset()
    sample_images = []
    sample_paths = []
    
    for i in range(min(num_samples, test_generator.samples)):
        x_batch, y_batch = next(test_generator)
        sample_images.append(x_batch[0])
        sample_paths.append(test_generator.filepaths[i])
    
    # Find suitable convolutional layer for Grad-CAM
    last_conv_layers = ['conv_pw_13_relu', 'conv5_block16_concat', 'cbam_mobilenet', 'cbam_densenet']
    layer_name = None
    
    for layer in last_conv_layers:
        try:
            model.get_layer(layer)
            layer_name = layer
            print(f"Using layer for Grad-CAM: {layer_name}")
            break
        except:
            continue
    
    if layer_name is None:
        print("Could not find suitable convolutional layer for Grad-CAM")
        return
    
    for i, (img_array, img_path) in enumerate(zip(sample_images, sample_paths)):
        try:
            # Preprocess image
            img_for_gradcam = np.expand_dims(img_array, axis=0)
            
            # Get prediction
            preds = model.predict(img_for_gradcam, verbose=0)
            pred_class = np.argmax(preds[0])
            pred_prob = np.max(preds[0])
            true_class = test_generator.classes[i]
            
            # Generate heatmap for Grad-CAM
            heatmap, _ = get_gradcam_heatmap(img_for_gradcam, model, layer_name, pred_class)
            
            # Create base filename
            base_filename = f"{dataset_name.replace(' ', '_')}_sample_{i}_true_{true_class}_pred_{pred_class}"
            
            # 1. Original image
            original_img = cv2.imread(img_path)
            original_img = cv2.resize(original_img, input_shape[:2])
            cv2.imwrite(os.path.join(detection_dir, f"{base_filename}_original.png"), original_img)
            
            # 2. Grad-CAM detection boxes (Green)
            gradcam_bbox_path = os.path.join(detection_dir, f"{base_filename}_gradcam_boxes.png")
            create_gradcam_bounding_box(img_path, heatmap, gradcam_bbox_path)
            
            # 3. Traditional detection boxes (Blue)
            traditional_bbox_path = os.path.join(detection_dir, f"{base_filename}_traditional_boxes.png")
            create_traditional_bounding_box(img_path, traditional_bbox_path)
            
            # 4. Combined boxes (Green + Blue)
            combined_bbox_path = os.path.join(detection_dir, f"{base_filename}_combined_boxes.png")
            create_combined_bounding_boxes(img_path, heatmap, combined_bbox_path)
            
            # 5. Grad-CAM heatmap only
            plt.figure(figsize=(8, 6))
            plt.imshow(heatmap, cmap='jet')
            plt.title(f'Grad-CAM Heatmap\nTrue: {true_class}, Pred: {pred_class}, Prob: {pred_prob:.2f}')
            plt.colorbar()
            plt.savefig(os.path.join(detection_dir, f"{base_filename}_heatmap.png"), 
                       dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Generated detection boxes for sample {i}")
            
        except Exception as e:
            print(f"Error processing image {i}: {e}")
            continue
    
    print(f"Detection box visualizations saved to: {detection_dir}")

# ==================== COMPARISON ANALYSIS FUNCTION ====================
def analyze_detection_comparison(model, test_generator, output_dir, dataset_name, counter, num_samples=10):
    """Analyze and compare Grad-CAM vs traditional detection methods"""
    comparison_dir = os.path.join(output_dir, 'detection_comparison')
    os.makedirs(comparison_dir, exist_ok=True)
    
    results = []
    test_generator.reset()
    
    # Find suitable convolutional layer for Grad-CAM
    last_conv_layers = ['conv_pw_13_relu', 'conv5_block16_concat', 'cbam_mobilenet', 'cbam_densenet']
    layer_name = None
    
    for layer in last_conv_layers:
        try:
            model.get_layer(layer)
            layer_name = layer
            break
        except:
            continue
    
    if layer_name is None:
        print("Could not find suitable convolutional layer for comparison analysis")
        return
    
    for i in range(min(num_samples, test_generator.samples)):
        try:
            x_batch, y_batch = next(test_generator)
            img_array = x_batch[0]
            img_path = test_generator.filepaths[i]
            
            # Preprocess for Grad-CAM
            img_for_gradcam = np.expand_dims(img_array, axis=0)
            heatmap, _ = get_gradcam_heatmap(img_for_gradcam, model, layer_name)
            
            # Count Grad-CAM boxes
            gradcam_boxes = count_bounding_boxes_gradcam(heatmap)
            
            # Count traditional boxes
            traditional_boxes = count_bounding_boxes_traditional(img_path)
            
            results.append({
                'sample_id': i,
                'gradcam_boxes': gradcam_boxes,
                'traditional_boxes': traditional_boxes,
                'difference': abs(gradcam_boxes - traditional_boxes)
            })
            
        except Exception as e:
            print(f"Error in comparison analysis for sample {i}: {e}")
            continue
    
    # Save comparison results
    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(os.path.join(comparison_dir, f'detection_comparison_{counter}.csv'), index=False)
    
    # Create comparison plot
    plt.figure(figsize=(12, 6))
    x = range(len(results))
    plt.bar(x, [r['gradcam_boxes'] for r in results], alpha=0.7, label='Grad-CAM Boxes', color='green')
    plt.bar(x, [r['traditional_boxes'] for r in results], alpha=0.7, label='Traditional Boxes', color='blue')
    plt.xlabel('Sample Index')
    plt.ylabel('Number of Detection Boxes')
    plt.title('Grad-CAM vs Traditional Detection Box Comparison')
    plt.legend()
    plt.xticks(x)
    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, f'comparison_plot_{counter}.png'), dpi=300)
    plt.close()
    
    print(f"Comparison analysis saved to: {comparison_dir}")

def count_bounding_boxes_gradcam(heatmap, threshold=0.5):
    """Count number of bounding boxes from Grad-CAM heatmap"""
    heatmap_resized = cv2.resize(heatmap, input_shape[:2])
    _, thresh = cv2.threshold(heatmap_resized, threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return len([c for c in contours if cv2.contourArea(c) > 10])

def count_bounding_boxes_traditional(img_path):
    """Count number of bounding boxes from traditional method"""
    img = cv2.imread(img_path)
    img = cv2.resize(img, input_shape[:2])
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, 50, 150)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    combined = cv2.bitwise_or(edges, morph)
    
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return len([c for c in contours if cv2.contourArea(c) > 50])

# ==================== MODIFY MAIN TRAINING LOOP ====================
# In your main training loop, after model evaluation, add:
print("Generating detection box visualizations...")
visualize_detection_boxes(model, test_generator, output_dir, dataset_name, counter, num_samples=2)

print("Running detection method comparison...")
analyze_detection_comparison(model, test_generator, output_dir, dataset_name, counter, num_samples=2)

Generating detection box visualizations...
Using layer for Grad-CAM: cbam_mobilenet
Error processing image 0: name 'get_gradcam_heatmap' is not defined
Error processing image 1: name 'get_gradcam_heatmap' is not defined
Detection box visualizations saved to: E:\CombineData001_002\SplitData1\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput\detection_boxes
Running detection method comparison...
Error in comparison analysis for sample 0: name 'get_gradcam_heatmap' is not defined
Error in comparison analysis for sample 1: name 'get_gradcam_heatmap' is not defined
Comparison analysis saved to: E:\CombineData001_002\SplitData1\hybrid_MobDenseNetwithCBAMattention_Output_using_tensorflow\balanceoutput\detection_comparison


Xception Model 

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, ReLU, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report, accuracy_score, precision_recall_curve
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import matplotlib.cm as cm
import seaborn as sns
import cv2
import os
import json
import time
import csv
from tqdm import tqdm
import random
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

tf.random.set_seed(42)
np.random.seed(42)

IMG_SIZE_DEFAULT = (224, 224)
KERNEL_INITIALIZER = "glorot_uniform"
BATCH_SIZE = 64
EPOCHS_INITIAL = 100
FORCE_TRAIN = True
LEARNING_RATES = [0.002]
DROPOUT_RATES = [0.2]
MODEL_NAME = 'Xception'
LAST_CONV_LAYER_NAME = 'block14_sepconv2_act'

# Define datasets with directory paths (generators will be set dynamically)
DATASETS = [
    {
        "name": "Imbalanced Dataset 1",
        "train_dir": r"E:\Data001\SplitData\train",
        "test_dir": r"E:\Data001\SplitData\test",
        "output_dir": r"E:\Data001\SplitData\xception_Output_using_tensorflow",
        "class_names": None,  
        "num_classes": None   
    },
    {
        "name": "Imbalanced Dataset 2",
        "train_dir": r"E:\Data002\SplitData\train",
        "test_dir": r"E:\Data002\SplitData\test",
        "output_dir": r"E:\Data002\SplitData\xception_Output_using_tensorflow",
        "class_names": None,
        "num_classes": None
    },
    {
        "name": "Imbalanced Combined Dataset",
        "train_dir": r"E:\CombineData001_002\SplitData\train",
        "test_dir": r"E:\CombineData001_002\SplitData\test",
        "output_dir": r"E:\CombineData001_002\SplitData\xception_Output_using_tensorflow",
        "class_names": None,
        "num_classes": None
    },
    {
        "name": "Balanced Dataset 1",
        "train_dir": r"E:\Data001\SplitData1\train",
        "test_dir": r"E:\Data001\SplitData1\test",
        "output_dir": r"E:\Data001\SplitData1\xception_Output_using_tensorflow",
        "class_names": None,
        "num_classes": None
    },
    {
        "name": "Balanced Dataset 2",
        "train_dir": r"E:\Data002\SplitData1\train",
        "test_dir": r"E:\Data002\SplitData1\test",
        "output_dir": r"E:\Data002\SplitData1\xception_Output_using_tensorflow",
        "class_names": None,
        "num_classes": None
    },
    {
        "name": "Balanced Combined Dataset",
        "train_dir": r"E:\CombineData001_002\SplitData1\train",
        "test_dir": r"E:\CombineData001_002\SplitData1\test",
        "output_dir": r"E:\CombineData001_002\SplitData1\xception_Output_using_tensorflow",
        "class_names": None,
        "num_classes": None
    }
]

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f"Using device: {physical_devices}, Num GPUs: {len(physical_devices)}")

def get_img_array(img, size):
    array = keras.utils.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    try:
        grad_model = keras.models.Model(
            model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
        )
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]
        grads = tape.gradient(class_channel, last_conv_layer_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + 1e-8)
        return heatmap.numpy()
    except Exception as e:
        print(f"Error in make_gradcam_heatmap: {e}")
        return None

def load_and_preprocess_image(img_path):
    """Load and preprocess an image for Xception model."""
    try:
        img = tf.keras.utils.load_img(img_path, target_size=IMG_SIZE_DEFAULT)
        img_array = tf.keras.utils.img_to_array(img)
        img_array = tf.keras.applications.xception.preprocess_input(img_array)
        return img_array
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

def create_model(dropout_rate, num_classes):
    try:
        base_model = Xception(weights='imagenet', include_top=False, input_shape=(*IMG_SIZE_DEFAULT, 3))
        base_model.trainable = False
        for layer in base_model.layers[-20:]:
            layer.trainable = True
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, kernel_initializer=KERNEL_INITIALIZER)(x)
        x = ReLU()(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout_rate)(ReLU()(x))
        x = Dense(128, kernel_initializer=KERNEL_INITIALIZER)(x)
        x = ReLU()(x)
        x = Dropout(dropout_rate)(x)
        outputs = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)
        model = Model(inputs=base_model.input, outputs=outputs)
        return model
    except Exception as e:
        print(f"Error in create_model: {e}")
        raise

def compute_metrics(y_true, y_pred, dataset_name):
    try:
        print(f"Computing {dataset_name} metrics: y_true shape={np.array(y_true).shape}, y_pred shape={np.array(y_pred).shape}")
        print(f"y_true sample: {y_true[:5]}, y_pred sample: {y_pred[:5]}")
        precision = precision_score(y_true, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_true, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_true, y_pred, average='weighted', zero_division=0)
        print(f"{dataset_name} metrics - Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
        return {
            f"{dataset_name}_precision": precision,
            f"{dataset_name}_recall": recall,
            f"{dataset_name}_f1": f1
        }
    except Exception as e:
        print(f"Error in compute_metrics: {e}")
        return {f"{dataset_name}_precision": 0, f"{dataset_name}_recall": 0, f"{dataset_name}_f1": 0}

def train_model(model, learning_rate, train_generator, test_generator, class_weights, counter, output_dir):
    try:
        optimizer = Nadam(learning_rate=learning_rate)
        model.compile(
            optimizer=optimizer,
            loss='categorical_crossentropy',
            metrics=['accuracy']
        )
        print(f"Model compiled with optimizer: {model.optimizer}")

        reduce_lr = ReduceLROnPlateau(factor=0.5, patience=3, monitor='val_loss')
        early_stopping = EarlyStopping(patience=10, monitor='val_loss', restore_best_weights=True)
        csv_logger = CSVLogger(
            os.path.join(output_dir, f'{MODEL_NAME}_metrics_per_epoch_{counter}.csv'),
            separator=',', append=True
        )

        class TimeHistory(keras.callbacks.Callback):
            def on_train_begin(self, logs={}):
                self.times = []
            def on_epoch_begin(self, epoch, logs={}):
                self.epoch_time_start = time.time()
            def on_epoch_end(self, epoch, logs={}):
                self.times.append(time.time() - self.epoch_time_start)

        time_callback = TimeHistory()

        history = model.fit(
            train_generator,
            epochs=EPOCHS_INITIAL,
            validation_data=test_generator,
            class_weight=class_weights,
            callbacks=[reduce_lr, early_stopping, csv_logger, time_callback],
            verbose=1
        )

        history_dict = {
            'train_loss': history.history['loss'],
            'train_acc': history.history['accuracy'],
            'val_loss': history.history.get('val_loss', []),
            'val_acc': history.history.get('val_accuracy', []),
            'time': time_callback.times
        }

        metrics_log = {'train_precision': [], 'train_recall': [], 'train_f1': [],
                       'test_precision': [], 'test_recall': [], 'test_f1': []}
        
        for epoch in range(len(history_dict['train_loss'])):
            train_preds, train_labels = [], []
            train_generator.reset()
            print(f"Epoch {epoch + 1}: Collecting training predictions")
            for batch_x, batch_y in train_generator:
                preds = np.argmax(model.predict(batch_x, verbose=0), axis=1)
                true_labels = np.argmax(batch_y, axis=1)
                train_preds.extend(preds)
                train_labels.extend(true_labels)
                if len(train_preds) >= train_generator.samples:
                    break
            train_metrics = compute_metrics(train_labels, train_preds, 'train')
            metrics_log['train_precision'].append(train_metrics['train_precision'])
            metrics_log['train_recall'].append(train_metrics['train_recall'])
            metrics_log['train_f1'].append(train_metrics['train_f1'])

            test_preds, test_labels = [], []
            test_generator.reset()
            print(f"Epoch {epoch + 1}: Collecting test predictions")
            for batch_x, batch_y in test_generator:
                preds = np.argmax(model.predict(batch_x, verbose=0), axis=1)
                true_labels = np.argmax(batch_y, axis=1)
                test_preds.extend(preds)
                test_labels.extend(true_labels)
                if len(test_preds) >= test_generator.samples:
                    break
            test_metrics = compute_metrics(test_labels, test_preds, 'test')
            metrics_log['test_precision'].append(test_metrics['test_precision'])
            metrics_log['test_recall'].append(test_metrics['test_recall'])
            metrics_log['test_f1'].append(test_metrics['test_f1'])

            csv_path = os.path.join(output_dir, f'{MODEL_NAME}_metrics_per_epoch_{counter}.csv')
            if FORCE_TRAIN and epoch == 0 and os.path.exists(csv_path):
                os.remove(csv_path) 
            with open(csv_path, 'a', newline='') as csvfile:
                fieldnames = [
                    'epoch', 'time', 'train/loss', 'train/accuracy', 'train/precision', 'train/recall', 'train/f1',
                    'test/loss', 'test/accuracy', 'test/precision', 'test/recall', 'test/f1', 'lr'
                ]
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                if csvfile.tell() == 0:
                    writer.writeheader()
                writer.writerow({
                    'epoch': epoch + 1,
                    'time': history_dict['time'][epoch],
                    'train/loss': history_dict['train_loss'][epoch],
                    'train/accuracy': history_dict['train_acc'][epoch],
                    'train/precision': train_metrics['train_precision'],
                    'train/recall': train_metrics['train_recall'],
                    'train/f1': train_metrics['train_f1'],
                    'test/loss': history_dict['val_loss'][epoch] if epoch < len(history_dict['val_loss']) else 0,
                    'test/accuracy': history_dict['val_acc'][epoch] if epoch < len(history_dict['val_acc']) else 0,
                    'test/precision': test_metrics['test_precision'],
                    'test/recall': test_metrics['test_recall'],
                    'test/f1': test_metrics['test_f1'],
                    'lr': tf.keras.backend.get_value(model.optimizer.learning_rate)
                })
                history_dict = {
            'train_loss': history.history['loss'],
            'train_acc': history.history['accuracy'],
            'val_loss': history.history.get('val_loss', []),
            'val_acc': history.history.get('val_accuracy', []),
            'time': time_callback.times
        }

        metrics_log = {'train_precision': [], 'train_recall': [], 'train_f1': [],
                       'test_precision': [], 'test_recall': [], 'test_f1': []}

        model.save_weights(os.path.join(output_dir, f'{MODEL_NAME}_{counter}.weights.h5'))
        
        with open(os.path.join(output_dir, f'{MODEL_NAME}_{counter}_history.json'), 'w') as f:
            json.dump(history_dict, f)
        
        print(f"Training completed. Epoch times: {history_dict['time']}")
        return history_dict, metrics_log
    except Exception as e:
        print(f"Error in train_model: {e}")
        raise

def evaluate_model(model, test_generator, class_names, counter):
    try:
        test_loss, test_acc = 0, 0
        test_preds, test_labels, test_probs, test_images, test_filenames = [], [], [], [], []
        start_time = time.time()
        
        test_generator.reset()
        for batch_x, batch_y in test_generator:
            loss, acc = model.test_on_batch(batch_x, batch_y)
            test_loss += loss * batch_x.shape[0]
            test_acc += acc * batch_x.shape[0]
            preds = np.argmax(model.predict(batch_x, verbose=0), axis=1)
            true_labels = np.argmax(batch_y, axis=1)
            probs = model.predict(batch_x, verbose=0)
            test_preds.extend(preds)
            test_labels.extend(true_labels)
            test_probs.extend(probs)
            test_images.extend(batch_x)
            test_filenames.extend(test_generator.filenames)
            if len(test_preds) >= test_generator.samples:
                break
        
        test_loss /= test_generator.samples
        test_acc /= test_generator.samples
        inference_time = (time.time() - start_time) * 1000
        
        test_probs = np.array(test_probs)
        if np.any(np.isnan(test_probs)) or np.any(test_probs < 0):
            print("Warning: test_probs contains NaN or negative values")
            test_probs = np.nan_to_num(test_probs, nan=0.0, posinf=1.0, neginf=0.0)
        row_sums = test_probs.sum(axis=1, keepdims=True)
        row_sums[row_sums == 0] = 1.0
        test_probs = test_probs / row_sums
        
        test_precision = precision_score(test_labels, test_preds, average='weighted', zero_division=0)
        test_recall = recall_score(test_labels, test_preds, average='weighted', zero_division=0)
        test_f1 = f1_score(test_labels, test_preds, average='weighted', zero_division=0)
        auc = None
        if len(np.unique(test_labels)) > 1:
            try:
                auc = roc_auc_score(test_labels, test_probs, multi_class='ovr')
            except ValueError as e:
                print(f"ROC AUC calculation failed: {e}")
                auc = None
        precision_per_class = precision_score(test_labels, test_preds, average=None, zero_division=0)
        recall_per_class = recall_score(test_labels, test_preds, average=None, zero_division=0)
        
        auc_display = f"{auc:.4f}" if auc is not None else 'N/A'
        print(f"Test Results: Loss={test_loss:.4f}, Acc={test_acc:.4f}, "
              f"Precision={test_precision:.4f}, Recall={test_recall:.4f}, F1={test_f1:.4f}, "
              f"AUC={auc_display}, Inference Time={inference_time:.2f}ms")
        print(f"Per-class Precision: {dict(zip(class_names, precision_per_class))}")
        print(f"Per-class Recall: {dict(zip(class_names, recall_per_class))}")
        
        return {
            'Model with input image size': MODEL_NAME,
            'Accuracy': test_acc,
            'AUC': auc,
            'Loss': test_loss,
            'Precision': test_precision,
            'Recall': test_recall,
            'F1 Score': test_f1,
            'Inference time (in miliseconds)': inference_time
        }, test_labels, test_preds, test_images, test_filenames
    except Exception as e:
        print(f"Error in evaluate_model: {e}")
        raise

def run_gradcam_detection(model, images, true_labels, pred_labels, filenames, class_names, counter, last_conv_layer_name, output_dir, num_images=6):
    try:
        detected_boxes = []
        detected_images = []
        
        for img, true_label, pred_label, fname in zip(images[:num_images], true_labels[:num_images], pred_labels[:num_images], filenames[:num_images]):
            img_array = get_img_array(img, size=IMG_SIZE_DEFAULT)
            heatmap = make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=pred_label)
            if heatmap is None:
                print(f"Skipping image {fname} due to heatmap generation failure")
                continue
            heatmap = (heatmap - heatmap.min()) / (heatmap.max() - heatmap.min() + 1e-8)
            
            heatmap_resized = zoom(heatmap, (IMG_SIZE_DEFAULT[0] / heatmap.shape[0], IMG_SIZE_DEFAULT[1] / heatmap.shape[1]))
            heatmap_resized = np.clip(heatmap_resized, 0, 1)
            
            threshold = 0.7
            binary_map = heatmap_resized > threshold
            if not np.any(binary_map):
                print(f"No significant regions detected for {fname}")
                continue
            
            coords = np.where(binary_map)
            if len(coords[0]) == 0:
                print(f"No coordinates found for {fname}")
                continue
            x_min, x_max = coords[1].min(), coords[1].max()
            y_min, y_max = coords[0].min(), coords[0].max()
            x_min, x_max = x_min * (IMG_SIZE_DEFAULT[0] / heatmap_resized.shape[1]), x_max * (IMG_SIZE_DEFAULT[0] / heatmap_resized.shape[1])
            y_min, y_max = y_min * (IMG_SIZE_DEFAULT[0] / heatmap_resized.shape[0]), y_max * (IMG_SIZE_DEFAULT[0] / heatmap_resized.shape[0])
            
            detected_boxes.append({
                "filename": fname,
                "boxes": [[x_min, y_min, x_max, y_max]],
                "labels": [class_names[pred_label]],
                "scores": [heatmap_resized.max()]
            })
            
            img_denorm = img * np.array([0.229, 0.224, 0.225]) * 255 + np.array([0.485, 0.456, 0.406]) * 255
            img_denorm = np.clip(img_denorm, 0, 255).astype(np.uint8)
            detected_images.append(img_denorm)
            detected_images[-1] = np.concatenate([detected_images[-1], heatmap_resized[..., None] * 255], axis=-1).astype(np.uint8)
        
        plt.figure(figsize=(15, 10), dpi=300)
        for i, (img, detection) in enumerate(zip(detected_images, detected_boxes)):
            plt.subplot(2, 3, i + 1)
            heatmap = img[..., 3] / 255.0
            img_rgb = img[..., :3]
            heatmap = cm.jet(heatmap)[..., :3] * 255
            overlay = (0.5 * img_rgb + 0.5 * heatmap).astype(np.uint8)
            plt.imshow(overlay)
            for box, label, score in zip(detection["boxes"], detection["labels"], detection["scores"]):
                x_min, y_min, x_max, y_max = box
                plt.gca().add_patch(plt.Rectangle((x_min, y_min), x_max - x_min, y_max - y_min,
                                                 edgecolor='red', facecolor='none', linewidth=2))
                plt.text(x_min, y_min - 5, f"{label} ({score:.2f})", color='red', fontsize=8, bbox=dict(facecolor='white', alpha=0.7))
            true_label = class_names[true_labels[i]]
            pred_label = class_names[pred_labels[i]]
            plt.title(f"File: {os.path.basename(detection['filename'])}\nActual: {true_label}\nPredicted: {pred_label}", fontsize=10)
            plt.axis('off')
        
        plt.tight_layout()
        output_path = os.path.join(output_dir, f'{MODEL_NAME}_gradcam_detections_{counter}.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight', format='png')
        print(f"Saved Grad-CAM detection plot to: {output_path}")
        plt.close()
        
        return detected_boxes
    except Exception as e:
        print(f"Error in run_gradcam_detection: {e}")
        return []

def create_gradcam_bounding_box(img_path, heatmap, output_path, threshold=0.5):
    """Create bounding boxes based on Grad-CAM heatmap activation."""
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    img = cv2.resize(img, IMG_SIZE_DEFAULT)
    
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    _, thresh = cv2.threshold(heatmap, threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 10:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    cv2.imwrite(output_path, img)
    return img

def create_traditional_bounding_box(img_path, output_path):
    """Create bounding boxes using traditional image processing methods."""
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    img = cv2.resize(img, IMG_SIZE_DEFAULT)
    img_copy = img.copy()
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    combined = cv2.bitwise_or(edges, morph)
    
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours:
        if cv2.contourArea(contour) > 50:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    cv2.imwrite(output_path, img_copy)
    return img_copy

def create_combined_bounding_boxes(img_path, heatmap, output_path, gradcam_threshold=0.5):
    """Create image with both Grad-CAM and traditional bounding boxes."""
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    img = cv2.resize(img, IMG_SIZE_DEFAULT)
    img_copy = img.copy()
    
    heatmap_resized = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    _, thresh = cv2.threshold(heatmap_resized, gradcam_threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    contours_gradcam, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours_gradcam:
        if cv2.contourArea(contour) > 10:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (0, 255, 0), 2)
    
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    edges = cv2.Canny(gray, 50, 150)
    _, thresh_trad = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh_trad, cv2.MORPH_CLOSE, kernel)
    combined = cv2.bitwise_or(edges, morph)
    contours_trad, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    for contour in contours_trad:
        if cv2.contourArea(contour) > 50:
            x, y, w, h = cv2.boundingRect(contour)
            cv2.rectangle(img_copy, (x, y), (x + w, y + h), (255, 0, 0), 2)
    
    cv2.putText(img_copy, "Grad-CAM Boxes (Green)", (10, 30), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)
    cv2.putText(img_copy, "Traditional Boxes (Blue)", (10, 60), 
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 0, 0), 2)
    
    cv2.imwrite(output_path, img_copy)
    return img_copy

def visualize_detection_boxes(model, test_generator, output_dir, dataset_name, counter, num_samples=5):
    """Visualize both Grad-CAM and traditional detection boxes."""
    detection_dir = os.path.join(output_dir, 'detection_boxes')
    os.makedirs(detection_dir, exist_ok=True)
    
    test_generator.reset()
    sample_images = []
    sample_paths = []
    
    for i in range(min(num_samples, test_generator.samples)):
        x_batch, y_batch = next(test_generator)
        sample_images.append(x_batch[0])
        sample_paths.append(os.path.join(test_generator.directory, test_generator.filenames[i]))
    
    for i, (img_array, img_path) in enumerate(zip(sample_images, sample_paths)):
        try:
            img_for_gradcam = np.expand_dims(img_array, axis=0)
            preds = model.predict(img_for_gradcam, verbose=0)
            pred_class = np.argmax(preds[0])
            pred_prob = np.max(preds[0])
            true_class = test_generator.classes[i]
            
            heatmap = make_gradcam_heatmap(img_for_gradcam, model, LAST_CONV_LAYER_NAME, pred_class)
            if heatmap is None:
                print(f"Skipping image {img_path} due to heatmap generation failure")
                continue
            
            base_filename = f"{dataset_name.replace(' ', '_')}_sample_{i}_true_{true_class}_pred_{pred_class}"
            
            original_img = cv2.imread(img_path)
            if original_img is None:
                print(f"Failed to load image: {img_path}")
                continue
            original_img = cv2.resize(original_img, IMG_SIZE_DEFAULT)
            cv2.imwrite(os.path.join(detection_dir, f"{base_filename}_original.png"), original_img)
            
            gradcam_bbox_path = os.path.join(detection_dir, f"{base_filename}_gradcam_boxes.png")
            create_gradcam_bounding_box(img_path, heatmap, gradcam_bbox_path)
            
            traditional_bbox_path = os.path.join(detection_dir, f"{base_filename}_traditional_boxes.png")
            create_traditional_bounding_box(img_path, traditional_bbox_path)
            
            combined_bbox_path = os.path.join(detection_dir, f"{base_filename}_combined_boxes.png")
            create_combined_bounding_boxes(img_path, heatmap, combined_bbox_path)
            
            plt.figure(figsize=(8, 6))
            plt.imshow(heatmap, cmap='jet')
            plt.title(f'Grad-CAM Heatmap\nTrue: {true_class}, Pred: {pred_class}, Prob: {pred_prob:.2f}')
            plt.colorbar()
            plt.savefig(os.path.join(detection_dir, f"{base_filename}_heatmap.png"), 
                       dpi=300, bbox_inches='tight')
            plt.close()
            
            print(f"Generated detection boxes for sample {i}")
            
        except Exception as e:
            print(f"Error processing image {i}: {e}")
            continue
    
    print(f"Detection box visualizations saved to: {detection_dir}")

def count_bounding_boxes_gradcam(heatmap, threshold=0.5):
    """Count number of bounding boxes from Grad-CAM heatmap."""
    heatmap_resized = cv2.resize(heatmap, IMG_SIZE_DEFAULT)
    _, thresh = cv2.threshold(heatmap_resized, threshold, 1.0, cv2.THRESH_BINARY)
    thresh = (thresh * 255).astype(np.uint8)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return len([c for c in contours if cv2.contourArea(c) > 10])

def count_bounding_boxes_traditional(img_path):
    """Count number of bounding boxes from traditional method."""
    img = cv2.imread(img_path)
    if img is None:
        raise ValueError(f"Failed to load image: {img_path}")
    img = cv2.resize(img, IMG_SIZE_DEFAULT)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    edges = cv2.Canny(gray, 50, 150)
    _, thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    kernel = np.ones((5, 5), np.uint8)
    morph = cv2.morphologyEx(thresh, cv2.MORPH_CLOSE, kernel)
    combined = cv2.bitwise_or(edges, morph)
    
    contours, _ = cv2.findContours(combined, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    
    return len([c for c in contours if cv2.contourArea(c) > 50])

def analyze_detection_comparison(model, test_generator, output_dir, dataset_name, counter, num_samples=10):
    """Analyze and compare Grad-CAM vs traditional detection methods."""
    comparison_dir = os.path.join(output_dir, 'detection_comparison')
    os.makedirs(comparison_dir, exist_ok=True)
    
    results = []
    test_generator.reset()
    
    for i in range(min(num_samples, test_generator.samples)):
        try:
            x_batch, y_batch = next(test_generator)
            img_array = x_batch[0]
            img_path = os.path.join(test_generator.directory, test_generator.filenames[i])
            
            img_for_gradcam = np.expand_dims(img_array, axis=0)
            preds = model.predict(img_for_gradcam, verbose=0)
            pred_class = np.argmax(preds[0])
            
            heatmap = make_gradcam_heatmap(img_for_gradcam, model, LAST_CONV_LAYER_NAME, pred_class)
            if heatmap is None:
                print(f"Skipping sample {i} due to heatmap generation failure")
                continue
            
            gradcam_boxes = count_bounding_boxes_gradcam(heatmap)
            traditional_boxes = count_bounding_boxes_traditional(img_path)
            
            results.append({
                'sample_id': i,
                'gradcam_boxes': gradcam_boxes,
                'traditional_boxes': traditional_boxes,
                'difference': abs(gradcam_boxes - traditional_boxes)
            })
            
        except Exception as e:
            print(f"Error in comparison analysis for sample {i}: {e}")
            continue
    
    comparison_df = pd.DataFrame(results)
    comparison_df.to_csv(os.path.join(comparison_dir, f'detection_comparison_{counter}.csv'), index=False)
    
    plt.figure(figsize=(12, 6))
    x = range(len(results))
    plt.bar(x, [r['gradcam_boxes'] for r in results], alpha=0.7, label='Grad-CAM Boxes', color='green')
    plt.bar(x, [r['traditional_boxes'] for r in results], alpha=0.7, label='Traditional Boxes', color='blue')
    plt.xlabel('Sample Index')
    plt.ylabel('Number of Detection Boxes')
    plt.title('Grad-CAM vs Traditional Detection Box Comparison')
    plt.legend()
    plt.xticks(x)
    plt.tight_layout()
    plt.savefig(os.path.join(comparison_dir, f'comparison_plot_{counter}.png'), dpi=300)
    plt.close()
    
    print(f"Comparison analysis saved to: {comparison_dir}")

def plot_training_history(history, metrics_log, output_dir, model_name, counter, class_names, best_y_true, best_y_pred):
    try:
        plt.figure(figsize=(18, 6), dpi=300)
        
        plt.subplot(1, 3, 1)
        plt.plot(range(1, len(history['train_acc']) + 1), history['train_acc'], label='Train Accuracy', 
                 color='#1f77b4', linewidth=2)
        plt.plot(range(1, len(history['val_acc']) + 1), history['val_acc'], label='Test Accuracy', 
                 color='#ff7f0e', linestyle='--', linewidth=2)
        plt.title('Train vs Test Accuracy', fontsize=14, pad=10)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.ylim(0, 1.05)
        
        plt.subplot(1, 3, 2)
        plt.plot(range(1, len(history['train_loss']) + 1), history['train_loss'], label='Train Loss', 
                 color='#1f77b4', linewidth=2)
        plt.plot(range(1, len(history['val_loss']) + 1), history['val_loss'], label='Test Loss', 
                 color='#ff7f0e', linestyle='--', linewidth=2)
        plt.title('Train vs Test Loss', fontsize=14, pad=10)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Loss', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.ylim(0, max(max(history['train_loss'], default=1), max(history['val_loss'], default=1)) * 1.1)
        
        plt.subplot(1, 3, 3)
        cm = confusion_matrix(best_y_true, best_y_pred, labels=range(len(class_names)))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                    xticklabels=class_names, yticklabels=class_names, 
                    cbar=False, square=True)
        plt.title('Test Confusion Matrix', fontsize=14, pad=10)
        plt.xlabel('Predicted', fontsize=12)
        plt.ylabel('True', fontsize=12)
        plt.xticks(rotation=45, ha='right', fontsize=10)
        plt.yticks(fontsize=10)
        
        plt.tight_layout(pad=2.0)
        output_path = os.path.join(output_dir, f'{MODEL_NAME}_training_metrics_{counter}.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight', format='png')
        print(f"Saved plot to: {output_path}")
        plt.close()
        
        for metric in ['precision', 'recall', 'f1']:
            plt.figure(figsize=(10, 5))
            plt.plot(range(1, len(metrics_log[f'train_{metric}']) + 1), metrics_log[f'train_{metric}'], 
                     label=f'Train {metric.capitalize()}', color='#1f77b4')
            plt.plot(range(1, len(metrics_log[f'test_{metric}']) + 1), metrics_log[f'test_{metric}'], 
                     label=f'Test {metric.capitalize()}', color='#ff7f0e', linestyle='--')
            plt.title(f'{MODEL_NAME} {metric.capitalize()}', fontsize=14)
            plt.ylabel(metric.capitalize(), fontsize=12)
            plt.xlabel('Epoch', fontsize=12)
            plt.legend(fontsize=10)
            plt.grid(True, alpha=0.3)
            plt.savefig(os.path.join(output_dir, f'{MODEL_NAME}_{metric}_{counter}.png'), dpi=300, bbox_inches='tight')
            plt.close()
        
        error_tipo_I, error_tipo_II = [], []
        for i in range(len(class_names)):
            FP = sum(cm[:, i]) - cm[i, i]
            FN = sum(cm[i, :]) - cm[i, i]
            TP = cm[i, i]
            TN = cm.sum() - (FP + FN + TP)
            FPR = FP / (FP + TN) if (FP + TN) > 0 else 0
            FNR = FN / (FN + TP) if (FN + TP) > 0 else 0
            error_tipo_I.append(FPR)
            error_tipo_II.append(FNR)
        
        df_errores = pd.DataFrame({
            'Clase': class_names,
            'Error Tipo I (FPR)': error_tipo_I,
            'Error Tipo II (FNR)': error_tipo_II
        })
        print("Classification Report:")
        print(classification_report(best_y_true, best_y_pred, target_names=class_names, digits=4, zero_division=0))
        pd.set_option('display.float_format', '{:.4g}'.format)
        print("\nErrores Tipo I y Tipo II por clase:")
        print(df_errores.to_string(index=False))
    except Exception as e:
        print(f"Error in plot_training_history: {e}")

def display_classification_images(images, true_labels, pred_labels, class_names, output_dir, counter, num_images=5):
    try:
        plt.figure(figsize=(15, 5), dpi=300)
        images = np.array(images)
        mean = np.array([0.485, 0.456, 0.406]) * 255
        std = np.array([0.229, 0.224, 0.225]) * 255
        images = images * std + mean
        images = np.clip(images, 0, 255).astype(np.uint8)
        
        for i in range(min(num_images, len(images))):
            plt.subplot(1, num_images, i + 1)
            plt.imshow(images[i])
            true_label = class_names[true_labels[i]]
            pred_label = class_names[pred_labels[i]]
            plt.title(f"Actual: {true_label}\nPredicted: {pred_label}", fontsize=10)
            plt.axis('off')
        
        plt.tight_layout()
        output_path = os.path.join(output_dir, f'{MODEL_NAME}_sample_predictions_{counter}.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight', format='png')
        print(f"Saved classification sample predictions plot to: {output_path}")
        plt.close()
    except Exception as e:
        print(f"Error in display_classification_images: {e}")

def save_chartjs_configs(history, output_dir, model_name, counter):
    try:
        loss_chart = {
            "type": "line",
            "data": {
                "labels": [str(i+1) for i in range(len(history['train_loss']))],
                "datasets": [
                    {
                        "label": "Train Loss",
                        "data": history['train_loss'],
                        "borderColor": "rgba(31, 119, 180, 1)",
                        "backgroundColor": "rgba(31, 119, 180, 0.2)",
                        "fill": False,
                        "tension": 0.4
                    },
                    {
                        "label": "Test Loss",
                        "data": history['val_loss'],
                        "borderColor": "rgba(255, 127, 14, 1)",
                        "backgroundColor": "rgba(255, 127, 14, 0.2)",
                        "fill": False,
                        "borderDash": [5, 5]
                    }
                ]
            },
            "options": {
                "responsive": True,
                "plugins": {
                    "legend": {"position": "top"},
                    "title": {"display": True, "text": "Train vs Test Loss"}
                },
                "scales": {
                    "x": {"title": {"display": True, "text": "Epoch"}},
                    "y": {"title": {"display": True, "text": "Loss"}, "beginAtZero": True}
                }
            }
        }
        accuracy_chart = {
            "type": "line",
            "data": {
                "labels": [str(i+1) for i in range(len(history['train_acc']))],
                "datasets": [
                    {
                        "label": "Train Accuracy",
                        "data": history['train_acc'],
                        "borderColor": "rgba(31, 119, 180, 1)",
                        "backgroundColor": "rgba(31, 119, 180, 0.2)",
                        "fill": False,
                        "tension": 0.4
                    },
                    {
                        "label": "Test Accuracy",
                        "data": history['val_acc'],
                        "borderColor": "rgba(255, 127, 14, 1)",
                        "backgroundColor": "rgba(255, 127, 14, 0.2)",
                        "fill": False,
                        "borderDash": [5, 5]
                    }
                ]
            },
            "options": {
                "responsive": True,
                "plugins": {
                    "legend": {"position": "top"},
                    "title": {"display": True, "text": "Train vs Test Accuracy"}
                },
                "scales": {
                    "x": {"title": {"display": True, "text": "Epoch"}},
                    "y": {"title": {"display": True, "text": "Accuracy"}, "beginAtZero": True, "max": 1}
                }
            }
        }
        with open(os.path.join(output_dir, f'{MODEL_NAME}_loss_chart_{counter}.json'), 'w') as f:
            json.dump(loss_chart, f, indent=2)
        with open(os.path.join(output_dir, f'{MODEL_NAME}_accuracy_chart_{counter}.json'), 'w') as f:
            json.dump(accuracy_chart, f, indent=2)
        print(f"Saved Chart.js configurations to: {output_dir}")
    except Exception as e:
        print(f"Error in save_chartjs_configs: {e}")

def plot_per_class_accuracy(model, train_generator, class_names, output_dir, counter, epochs):
    """
    Compute and visualize per-class accuracy on the training set for each epoch.
    
    Args:
        model: Trained Keras model.
        train_generator: Training data generator.
        class_names: List of class names.
        output_dir: Directory to save the plot and Chart.js config.
        counter: Counter for unique file naming.
        epochs: Number of epochs to evaluate.
    """
    try:
        per_class_accuracies = {class_name: [] for class_name in class_names}
        
        for epoch in range(epochs):
            train_generator.reset()
            y_true, y_pred = [], []
            print(f"Epoch {epoch + 1}: Collecting per-class training predictions")
            for batch_x, batch_y in train_generator:
                preds = np.argmax(model.predict(batch_x, verbose=0), axis=1)
                true_labels = np.argmax(batch_y, axis=1)
                y_true.extend(true_labels)
                y_pred.extend(preds)
                if len(y_true) >= train_generator.samples:
                    break
            
            y_true = np.array(y_true)
            y_pred = np.array(y_pred)
            
            for i, class_name in enumerate(class_names):
                class_mask = y_true == i
                if np.sum(class_mask) > 0:
                    class_acc = accuracy_score(y_true[class_mask], y_pred[class_mask])
                else:
                    class_acc = 0.0
                per_class_accuracies[class_name].append(class_acc)
        
        colors = ['rgba(31, 119, 180, 1)', 'rgba(255, 127, 14, 1)', 'rgba(44, 160, 44, 1)', 
                  'rgba(214, 39, 40, 1)', 'rgba(148, 103, 189, 1)']
        accuracy_chart_config = {
            "type": "line",
            "data": {
                "labels": [str(i+1) for i in range(epochs)],
                "datasets": [
                    {
                        "label": class_name,
                        "data": per_class_accuracies[class_name],
                        "borderColor": colors[i % len(colors)],
                        "backgroundColor": colors[i % len(colors)].replace('1)', '0.2)'),
                        "fill": False,
                        "tension": 0.4
                    } for i, class_name in enumerate(class_names)
                ]
            },
            "options": {
                "responsive": True,
                "plugins": {
                    "legend": {"position": "top"},
                    "title": {"display": True, "text": "Per-Class Training Accuracy"}
                },
                "scales": {
                    "x": {"title": {"display": True, "text": "Epoch"}},
                    "y": {"title": {"display": True, "text": "Accuracy"}, "min": 0, "max": 1}
                }
            }
        }
        
        plt.figure(figsize=(10, 6), dpi=300)
        for i, class_name in enumerate(class_names):
            plt.plot(range(1, epochs + 1), per_class_accuracies[class_name], 
                     label=f'{class_name}', color=colors[i % len(colors)])
        
        plt.title('Per-Class Training Accuracy', fontsize=14)
        plt.xlabel('Epoch', fontsize=12)
        plt.ylabel('Accuracy', fontsize=12)
        plt.legend(fontsize=10)
        plt.grid(True, alpha=0.3)
        plt.xlim(1, epochs)
        plt.ylim(0, 1)
        output_path = os.path.join(output_dir, f'{MODEL_NAME}_per_class_accuracy_{counter}.png')
        plt.savefig(output_path, dpi=300, bbox_inches='tight')
        plt.close()
        print(f"Saved per-class accuracy plot to: {output_path}")
        
        with open(os.path.join(output_dir, f'{MODEL_NAME}_per_class_accuracy_chart_{counter}.json'), 'w') as f:
            json.dump(accuracy_chart_config, f, indent=2)
        print(f"Saved per-class accuracy Chart.js config to: {output_dir}")
        
        return per_class_accuracies
    
    except Exception as e:
        print(f"Error in plot_per_class_accuracy: {e}")
        return {}

if __name__ == '__main__':
    try:
        print(f"TensorFlow version: {tf.__version__}")
        
        for dataset in DATASETS:
            dataset_name = dataset['name']
            TRAIN_DIR = dataset['train_dir']
            TEST_DIR = dataset['test_dir']
            OUTPUT_DIR = dataset['output_dir']
            
            print(f"\nProcessing {dataset_name}...")
            os.makedirs(OUTPUT_DIR, exist_ok=True)

            train_datagen = ImageDataGenerator(
                rotation_range=15,
                horizontal_flip=True,
                vertical_flip=True,
                zoom_range=[0.8, 1.2],
                width_shift_range=0.2,
                height_shift_range=0.2,
                preprocessing_function=tf.keras.applications.xception.preprocess_input
            )
            test_datagen = ImageDataGenerator(
                preprocessing_function=tf.keras.applications.xception.preprocess_input
            )
            train_generator = train_datagen.flow_from_directory(
                TRAIN_DIR,
                target_size=IMG_SIZE_DEFAULT,
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                shuffle=True
            )
            test_generator = test_datagen.flow_from_directory(
                TEST_DIR,
                target_size=IMG_SIZE_DEFAULT,
                batch_size=BATCH_SIZE,
                class_mode='categorical',
                shuffle=False
            )
            
            class_names = list(train_generator.class_indices.keys())
            NUM_CLASSES = len(class_names)
            print(f"[{dataset_name}] Detected {NUM_CLASSES} classes: {class_names}")
            
            if NUM_CLASSES < 2:
                raise ValueError(f"[{dataset_name}] Found {NUM_CLASSES} classes. At least 2 classes are required.")
            
            sample_inputs, sample_labels = next(train_generator)
            print(f"[{dataset_name}] Input shape for Xception: {sample_inputs.shape}, Label shape: {sample_labels.shape}")
            
            train_classes = train_generator.classes
            class_weights = compute_class_weight('balanced', classes=np.unique(train_classes), y=train_classes)
            class_weights = dict(enumerate(class_weights))
            print(f"[{dataset_name}] Class weights: {dict(zip(class_names, class_weights.values()))}")
            
            test_labels = test_generator.classes
            print(f"[{dataset_name}] Test dataset size: {test_generator.samples}")
            print(f"[{dataset_name}] Test labels unique: {np.unique(test_labels)}")
            
            results = []
            counter = 0
            histories = []
            best_test_accuracy = 0
            best_y_true, best_y_pred, best_images, best_filenames = None, None, None, None
            
            specific_model_path = os.path.join(OUTPUT_DIR, 'Xception_1.weights.h5')
            
            for lr in LEARNING_RATES:
                for dr in DROPOUT_RATES:
                    counter += 1
                    model_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_{counter}.weights.h5')
                    history_json_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_{counter}_history.json')
                    
                    if counter == 1 and os.path.exists(specific_model_path):
                        print(f"[{dataset_name}] Loading pre-trained model for {MODEL_NAME} with lr={lr}, dr={dr}, counter={counter}")
                        model = create_model(dr, num_classes=NUM_CLASSES)
                        model.compile(
                            optimizer=Nadam(learning_rate=lr),
                            loss='categorical_crossentropy',
                            metrics=['accuracy']
                        )
                        model.load_weights(specific_model_path)
                        if os.path.exists(history_json_path):
                            with open(history_json_path, 'r') as f:
                                history_data = json.load(f)
                                histories.append((history_data, None))
                        else:
                            print(f"[{dataset_name}] Warning: History file {history_json_path} not found. Skipping plotting.")
                            histories.append((None, None))
                    elif os.path.exists(model_path) and not FORCE_TRAIN:
                        print(f"[{dataset_name}] Loading pre-trained model for {MODEL_NAME} with lr={lr}, dr={dr}, counter={counter}")
                        model = create_model(dr, num_classes=NUM_CLASSES)
                        model.compile(
                            optimizer=Nadam(learning_rate=lr),
                            loss='categorical_crossentropy',
                            metrics=['accuracy']
                        )
                        model.load_weights(model_path)
                        if os.path.exists(history_json_path):
                            with open(history_json_path, 'r') as f:
                                history_data = json.load(f)
                                histories.append((history_data, None))
                        else:
                            print(f"[{dataset_name}] Warning: History file {history_json_path} not found. Skipping plotting.")
                            histories.append((None, None))
                    else:
                        print(f"[{dataset_name}] Training {MODEL_NAME} with lr={lr}, dr={dr}, counter={counter}")
                        model = create_model(dr, num_classes=NUM_CLASSES)
                        history, metrics_log = train_model(model, lr, train_generator, test_generator, class_weights, counter, OUTPUT_DIR)
                        histories.append((history, metrics_log))
                        
                        print(f"[{dataset_name}] Computing per-class training accuracy...")
                        plot_per_class_accuracy(model, train_generator, class_names, OUTPUT_DIR, counter, len(history['train_loss']))
                    
                    result, y_true, y_pred, test_images, test_filenames = evaluate_model(model, test_generator, class_names, counter)
                    results.append(result)
                    
                    print(f"[{dataset_name}] Generating detection box visualizations...")
                    visualize_detection_boxes(model, test_generator, OUTPUT_DIR, dataset_name, counter, num_samples=2)
                    
                    print(f"[{dataset_name}] Running detection method comparison...")
                    analyze_detection_comparison(model, test_generator, OUTPUT_DIR, dataset_name, counter, num_samples=2)
                    
                    if result['Accuracy'] > best_test_accuracy:
                        best_test_accuracy = result['Accuracy']
                        best_y_true, best_y_pred, best_images, best_filenames = y_true, y_pred, test_images, test_filenames
                    
                    if best_y_true is not None and best_images is not None:
                        display_classification_images(best_images, best_y_true, best_y_pred, class_names, OUTPUT_DIR, counter)
                    
                    subfolders = class_names
                    selected_images = []
                    selected_true_labels = []
                    selected_pred_labels = []
                    selected_filenames = []
                    
                    for subfolder in subfolders:
                        subfolder_path = os.path.join(TEST_DIR, subfolder)
                        if not os.path.exists(subfolder_path):
                            print(f"[{dataset_name}] Warning: Subfolder {subfolder_path} does not exist")
                            continue
                        image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                        if len(image_files) < 2:
                            print(f"[{dataset_name}] Warning: Subfolder {subfolder} has fewer than 2 images ({len(image_files)})")
                            continue
                        selected_files = random.sample(image_files, 2)
                        class_idx = class_names.index(subfolder)
                        
                        for fname in selected_files:
                            img_path = os.path.join(subfolder_path, fname)
                            img_array = load_and_preprocess_image(img_path)
                            if img_array is None:
                                continue
                            pred = model.predict(np.expand_dims(img_array, axis=0), verbose=0)
                            pred_label = np.argmax(pred, axis=1)[0]
                            selected_images.append(img_array)
                            selected_true_labels.append(class_idx)
                            selected_pred_labels.append(pred_label)
                            selected_filenames.append(os.path.join(subfolder, fname))
                    
                    if selected_images:
                        detected_boxes = run_gradcam_detection(
                            model, selected_images, selected_true_labels, selected_pred_labels,
                            selected_filenames, class_names, counter, LAST_CONV_LAYER_NAME, OUTPUT_DIR, num_images=6
                        )
                    
                    if histories and histories[-1][0] is not None and best_y_true is not None:
                        history, metrics_log = histories[-1]
                        plot_training_history(history, metrics_log if metrics_log is not None else {
                            'train_precision': [], 'train_recall': [], 'train_f1': [],
                            'test_precision': [], 'test_recall': [], 'test_f1': []
                        }, OUTPUT_DIR, MODEL_NAME, counter, class_names, best_y_true, best_y_pred)
                        save_chartjs_configs(history, OUTPUT_DIR, MODEL_NAME, counter)
                    
                    tf.keras.backend.clear_session()
            
            with open(os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_results_{counter}.csv'), 'w', newline='') as csvfile:
                fieldnames = ['Model with input image size', 'Accuracy', 'AUC', 'Loss', 'Precision', 'Recall', 'F1 Score', 'Inference time (in miliseconds)']
                writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                writer.writeheader()
                for result in results:
                    writer.writerow(result)
            
            print(f"[{dataset_name}] Processing completed.")
            tf.keras.backend.clear_session()
    
    except KeyboardInterrupt:
        print("Training interrupted by user. Cleaning up...")
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    finally:
        tf.keras.backend.clear_session()

Using device: [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')], Num GPUs: 1
TensorFlow version: 2.10.1

Processing Imbalanced Dataset 1...
Found 2757 images belonging to 3 classes.
Found 307 images belonging to 3 classes.
[Imbalanced Dataset 1] Detected 3 classes: ['Glioma', 'Meningioma', 'Pituitary']
[Imbalanced Dataset 1] Input shape for Xception: (64, 224, 224, 3), Label shape: (64, 3)
[Imbalanced Dataset 1] Class weights: {'Glioma': 0.7162899454403742, 'Meningioma': 1.4427001569858713, 'Pituitary': 1.097968936678614}
[Imbalanced Dataset 1] Test dataset size: 307
[Imbalanced Dataset 1] Test labels unique: [0 1 2]
[Imbalanced Dataset 1] Training Xception with lr=0.002, dr=0.2, counter=1
Model compiled with optimizer: <keras.optimizers.optimizer_v2.nadam.Nadam object at 0x000002B34B4B4A90>
Epoch 1/100
44/44 [==============================] - 108s 2s/step - loss: 0.4790 - accuracy: 0.8143 - val_loss: 1.1033 - val_accuracy: 0.7752 - lr: 0.0020
Epoch 2/100
44/44 [======

ResourceExhaustedError: Graph execution error:

Detected at node 'model/block2_sepconv1/separable_conv2d' defined at (most recent call last):
    File "C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "e:\.env\lib\site-packages\ipykernel_launcher.py", line 18, in <module>
      app.launch_new_instance()
    File "e:\.env\lib\site-packages\traitlets\config\application.py", line 1075, in launch_instance
      app.start()
    File "e:\.env\lib\site-packages\ipykernel\kernelapp.py", line 739, in start
      self.io_loop.start()
    File "e:\.env\lib\site-packages\tornado\platform\asyncio.py", line 211, in start
      self.asyncio_loop.run_forever()
    File "C:\Users\LENOVO\AppData\Local\Programs\Python\Python310\lib\asyncio\base_events.py", line 5

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import Xception, MobileNet, DenseNet121,MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Dropout, ReLU, BatchNormalization, GlobalAveragePooling2D
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.callbacks import ReduceLROnPlateau, EarlyStopping, CSVLogger
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import roc_auc_score, confusion_matrix, precision_score, recall_score, f1_score, classification_report
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.ndimage import zoom
import matplotlib.cm as cm
import seaborn as sns
import os
import json
import time
import csv
from tqdm import tqdm
import random
from PIL import Image
import warnings
warnings.filterwarnings('ignore')

tf.random.set_seed(42)
np.random.seed(42)

# Configuration
IMG_SIZE_DEFAULT = (224, 224)
KERNEL_INITIALIZER = "glorot_uniform"
BATCH_SIZE = 64
EPOCHS_INITIAL = 80
FORCE_TRAIN = True
LEARNING_RATES = [0.002]
DROPOUT_RATES = [0.2]
DATASETS = [
    {
        'name': 'Dataset-1',
        'TRAIN_DIR': r'E:\dataset001\splitdata1\train',
        'TEST_DIR': r'E:\dataset001\splitdata1\test',
        'OUTPUT_DIR': r'E:\dataset001\splitdata1\output'
    },
    {
        'name': 'Dataset-2',
        'TRAIN_DIR': r'E:\dataset002\splitdata1\train',
        'TEST_DIR': r'E:\dataset002\splitdata1\test',
        'OUTPUT_DIR': r'E:\dataset002\splitdata1\output'
    }
]

# Define model configurations
MODEL_CONFIGS = [
    {
        'name': 'Xception',
        'base_model': Xception,
        'preprocess_fn': tf.keras.applications.xception.preprocess_input,
        'last_conv_layer': 'block14_sepconv2_act'
    },
        {
        'name': 'MobileNet',
        'base_model': MobileNet,
        'preprocess_fn': tf.keras.applications.mobilenet.process_input
        'last_conv_layer': 'block_16_project'
    },
    {
        'name': 'MobileNetV2',
        'base_model': MobileNetV2,
        'preprocess_fn': tf.keras.applications.mobilenet_v2.preprocess_input,
        'last_conv_layer': 'block_16_project'
    },
    {
        'name': 'DenseNet121',
        'base_model': DenseNet121,
        'preprocess_fn': tf.keras.applications.densenet.preprocess_input,
        'last_conv_layer': 'relu'
    }
]

physical_devices = tf.config.list_physical_devices('GPU')
if physical_devices:
    tf.config.experimental.set_memory_growth(physical_devices[0], True)
print(f"Using device: {physical_devices}, Num GPUs: {len(physical_devices)}")

def get_img_array(img, size):
    array = keras.utils.img_to_array(img)
    array = np.expand_dims(array, axis=0)
    return array

def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    try:
        grad_model = keras.models.Model(
            model.inputs, [model.get_layer(last_conv_layer_name).output, model.output]
        )
        with tf.GradientTape() as tape:
            last_conv_layer_output, preds = grad_model(img_array)
            if pred_index is None:
                pred_index = tf.argmax(preds[0])
            class_channel = preds[:, pred_index]
        grads = tape.gradient(class_channel, last_conv_layer_output)
        pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
        last_conv_layer_output = last_conv_layer_output[0]
        heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
        heatmap = tf.squeeze(heatmap)
        heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap + 1e-8)
        return heatmap.numpy()
    except Exception as e:
        print(f"Error in make_gradcam_heatmap: {e}")
        return None

def create_model(base_model_class, dropout_rate, num_classes, preprocess_fn):
    try:
        base_model = base_model_class(
            weights='imagenet',
            include_top=False,
            input_shape=(*IMG_SIZE_DEFAULT, 3)
        )
        base_model.trainable = False
        for layer in base_model.layers[-20:]:
            layer.trainable = True
        x = base_model.output
        x = GlobalAveragePooling2D()(x)
        x = Dense(512, kernel_initializer=KERNEL_INITIALIZER)(x)
        x = ReLU()(x)
        x = BatchNormalization()(x)
        x = Dropout(dropout_rate)(x)
        x = Dense(128, kernel_initializer=KERNEL_INITIALIZER)(x)
        x = ReLU()(x)
        x = Dropout(dropout_rate)(x)
        outputs = Dense(num_classes, activation='softmax', kernel_initializer='random_uniform')(x)
        model = Model(inputs=base_model.input, outputs=outputs)
        return model
    except Exception as e:
        print(f"Error in create_model: {e}")
        raise

def load_and_preprocess_image(img_path, preprocess_fn):
    try:
        img = Image.open(img_path).convert('RGB')
        img = img.resize(IMG_SIZE_DEFAULT)
        img_array = keras.utils.img_to_array(img)
        img_array = preprocess_fn(img_array)
        return img_array
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

# Rest of the functions (train_model, evaluate_model, etc.) remain unchanged

if __name__ == '__main__':
    try:
        print(f"TensorFlow version: {tf.__version__}")
        
        for dataset in DATASETS:
            dataset_name = dataset['name']
            TRAIN_DIR = dataset['TRAIN_DIR']
            TEST_DIR = dataset['TEST_DIR']
            OUTPUT_DIR = dataset['OUTPUT_DIR']
            
            print(f"\nProcessing {dataset_name}...")
            os.makedirs(OUTPUT_DIR, exist_ok=True)

            # Data generators
            for model_config in MODEL_CONFIGS:
                MODEL_NAME = model_config['name']
                preprocess_fn = model_config['preprocess_fn']
                train_datagen = ImageDataGenerator(
                    rotation_range=15,
                    horizontal_flip=True,
                    vertical_flip=True,
                    zoom_range=[0.8, 1.2],
                    width_shift_range=0.2,
                    height_shift_range=0.2,
                    preprocessing_function=preprocess_fn
                )
                test_datagen = ImageDataGenerator(
                    preprocessing_function=preprocess_fn
                )
                train_generator = train_datagen.flow_from_directory(
                    TRAIN_DIR,
                    target_size=IMG_SIZE_DEFAULT,
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    shuffle=True
                )
                test_generator = test_datagen.flow_from_directory(
                    TEST_DIR,
                    target_size=IMG_SIZE_DEFAULT,
                    batch_size=BATCH_SIZE,
                    class_mode='categorical',
                    shuffle=False
                )
                
                # Dynamically set NUM_CLASSES
                class_names = list(train_generator.class_indices.keys())
                NUM_CLASSES = len(class_names)
                print(f"[{dataset_name}] [{MODEL_NAME}] Detected {NUM_CLASSES} classes: {class_names}")
                
                if NUM_CLASSES < 2:
                    raise ValueError(f"[{dataset_name}] Found {NUM_CLASSES} classes. At least 2 classes are required.")
                
                train_classes = train_generator.classes
                class_weights = compute_class_weight('balanced', classes=np.unique(train_classes), y=train_classes)
                class_weights = dict(enumerate(class_weights))
                print(f"[{dataset_name}] [{MODEL_NAME}] Class weights: {dict(zip(class_names, class_weights.values()))}")
                
                results = []
                counter = 0
                histories = []
                best_test_accuracy = 0
                best_y_true, best_y_pred, best_images, best_filenames = None, None, None, None
                
                specific_model_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_1.weights.h5')
                
                for lr in LEARNING_RATES:
                    for dr in DROPOUT_RATES:
                        counter += 1
                        model_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_{counter}.weights.h5')
                        history_json_path = os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_{counter}_history.json')
                        
                        if counter == 1 and os.path.exists(specific_model_path):
                            print(f"[{dataset_name}] [{MODEL_NAME}] Loading pre-trained model with lr={lr}, dr={dr}, counter={counter}")
                            model = create_model(model_config['base_model'], dr, NUM_CLASSES, preprocess_fn)
                            model.compile(
                                optimizer=Nadam(learning_rate=lr),
                                loss='categorical_crossentropy',
                                metrics=['accuracy']
                            )
                            model.load_weights(specific_model_path)
                            if os.path.exists(history_json_path):
                                with open(history_json_path, 'r') as f:
                                    history_data = json.load(f)
                                    histories.append((history_data, None))
                            else:
                                print(f"[{dataset_name}] [{MODEL_NAME}] Warning: History file {history_json_path} not found.")
                                histories.append((None, None))
                        elif os.path.exists(model_path) and not FORCE_TRAIN:
                            print(f"[{dataset_name}] [{MODEL_NAME}] Loading pre-trained model with lr={lr}, dr={dr}, counter={counter}")
                            model = create_model(model_config['base_model'], dr, NUM_CLASSES, preprocess_fn)
                            model.compile(
                                optimizer=Nadam(learning_rate=lr),
                                loss='categorical_crossentropy',
                                metrics=['accuracy']
                            )
                            model.load_weights(model_path)
                            if os.path.exists(history_json_path):
                                with open(history_json_path, 'r') as f:
                                    history_data = json.load(f)
                                    histories.append((history_data, None))
                            else:
                                print(f"[{dataset_name}] [{MODEL_NAME}] Warning: History file {history_json_path} not found.")
                                histories.append((None, None))
                        else:
                            print(f"[{dataset_name}] [{MODEL_NAME}] Training with lr={lr}, dr={dr}, counter={counter}")
                            model = create_model(model_config['base_model'], dr, NUM_CLASSES, preprocess_fn)
                            history, metrics_log = train_model(model, lr, train_generator, test_generator, class_weights, counter)
                            histories.append((history, metrics_log))
                        
                        # Evaluate and get images
                        result, y_true, y_pred, test_images, test_filenames = evaluate_model(
                            model, test_generator, class_names, counter
                        )
                        results.append(result)
                        
                        if result['Accuracy'] > best_test_accuracy:
                            best_test_accuracy = result['Accuracy']
                            best_y_true, best_y_pred, best_images, best_filenames = y_true, y_pred, test_images, test_filenames
                        
                        # Display classification results
                        if best_y_true is not None and best_images is not None:
                            display_classification_images(
                                best_images, best_y_true, best_y_pred, class_names, OUTPUT_DIR, counter
                            )
                        
                        # Select 2 random images from each subfolder
                        subfolders = class_names
                        selected_images = []
                        selected_true_labels = []
                        selected_pred_labels = []
                        selected_filenames = []
                        
                        for subfolder in subfolders:
                            subfolder_path = os.path.join(TEST_DIR, subfolder)
                            if not os.path.exists(subfolder_path):
                                print(f"[{dataset_name}] [{MODEL_NAME}] Warning: Subfolder {subfolder_path} does not exist")
                                continue
                            image_files = [f for f in os.listdir(subfolder_path) if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
                            if len(image_files) < 2:
                                print(f"[{dataset_name}] [{MODEL_NAME}] Warning: Subfolder {subfolder} has fewer than 2 images")
                                continue
                            selected_files = random.sample(image_files, 2)
                            class_idx = class_names.index(subfolder)
                            
                            for fname in selected_files:
                                img_path = os.path.join(subfolder_path, fname)
                                img_array = load_and_preprocess_image(img_path, preprocess_fn)
                                if img_array is not None:
                                    pred = model.predict(np.expand_dims(img_array, axis=0), verbose=0)
                                    pred_label = np.argmax(pred, axis=1)[0]
                                    selected_images.append(img_array)
                                    selected_true_labels.append(class_idx)
                                    selected_pred_labels.append(pred_label)
                                    selected_filenames.append(os.path.join(subfolder, fname))
                        
                        if selected_images:
                            detected_boxes = run_gradcam_detection(
                                model, selected_images, selected_true_labels, selected_pred_labels,
                                selected_filenames, class_names, counter, model_config['last_conv_layer'], num_images=6
                            )
                        
                        if histories and histories[-1][0] is not None and best_y_true is not None:
                            history, metrics_log = histories[-1]
                            plot_training_history(
                                history, metrics_log if metrics_log is not None else {
                                    'train_precision': [], 'train_recall': [], 'train_f1': [],
                                    'test_precision': [], 'test_recall': [], 'test_f1': []
                                }, OUTPUT_DIR, MODEL_NAME, counter, class_names, best_y_true, best_y_pred
                            )
                            save_chartjs_configs(history, OUTPUT_DIR, MODEL_NAME, counter)
                        
                        tf.keras.backend.clear_session()
                
                with open(os.path.join(OUTPUT_DIR, f'{MODEL_NAME}_results_{counter}.csv'), 'w', newline='') as csvfile:
                    fieldnames = ['Model with input image size', 'Accuracy', 'AUC', 'Loss', 'Precision', 'Recall', 'F1 Score', 'Inference time (in miliseconds)']
                    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
                    writer.writeheader()
                    for result in results:
                        writer.writerow(result)
                
                print(f"[{dataset_name}] [{MODEL_NAME}] Processing completed.")
                tf.keras.backend.clear_session()
    
    except KeyboardInterrupt:
        print("Training interrupted by user. Cleaning up...")
    except Exception as e:
        print(f"An error occurred: {e}")
        raise
    finally:
        tf.keras.backend.clear_session()